<a href="https://colab.research.google.com/github/escenariosparalatransformacion/Chat-Rag/blob/main/LLM_Toolkit_for_Humanitarian_Negotiation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# @title Cell 1: Install Dependencies
# ---
# Execute this cell first to install necessary libraries.
# ---
print("⏳ Installing dependencies...")
# -U ensures packages are updated if already installed
!pip install -q -U gradio openai google-generativeai anthropic elevenlabs requests python-dotenv mistralai pandas numpy matplotlib pyttsx3
print("✅ Dependencies installed.")

# @title Cell 2: Utility Functions & API Calls (Real)
# ---
# Defines functions for Gradio components.
# Includes REAL API calls and instructions for local services (Ollama, n8n).
# All comments and docstrings are in English.
# ---
import gradio as gr
import os
import requests
import time
import base64
import json
import subprocess
from datetime import datetime
from dotenv import load_dotenv
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from pathlib import Path # For saving audio files
import pyttsx3 # For local text-to-speech (fallback)

# Load environment variables if a .env file exists (useful for local development)
load_dotenv()

# --- Cloud LLM API Call Functions (REAL) ---

def call_openai_api(api_key, prompt_text, model="gpt-4o-mini", system_message="You are a helpful assistant specialized in humanitarian negotiation analysis."):
    """
    Makes a real API call to OpenAI (ChatGPT).

    Args:
        api_key (str): OpenAI API key.
        prompt_text (str): The prompt to send to the model.
        model (str): The OpenAI model to use.
        system_message (str): System message to control assistant behavior.

    Returns:
        str: The response text from the model or an error message.
    """
    if not api_key:
        return "❌ Error: OpenAI API key not provided."
    try:
        from openai import OpenAI, AuthenticationError, APIError
        client = OpenAI(api_key=api_key)
        response = client.chat.completions.create(
            model=model,
            messages=[
                {"role": "system", "content": system_message},
                {"role": "user", "content": prompt_text}
            ]
        )
        return response.choices[0].message.content
    except ImportError:
        return "❌ Error: 'openai' library not found. Please ensure Cell 1 ran successfully."
    except AuthenticationError:
         return f"❌ OpenAI Authentication Error: Invalid API key or insufficient permissions."
    except APIError as e:
         return f"❌ OpenAI API Error: {e}"
    except Exception as e:
        return f"❌ An unexpected error occurred calling OpenAI: {e}"

def call_gemini_api(api_key, prompt_text, model="gemini-1.5-flash-latest"):
    """
    Makes a real API call to Google AI (Gemini).

    Args:
        api_key (str): Google AI API key.
        prompt_text (str): The prompt to send to the model.
        model (str): The Gemini model to use.

    Returns:
        str: The response text from the model or an error message.
    """
    if not api_key:
        return "❌ Error: Gemini API key not provided."
    try:
        import google.generativeai as genai
        from google.api_core import exceptions as google_exceptions # For specific errors

        genai.configure(api_key=api_key)
        gen_model = genai.GenerativeModel(model)
        response = gen_model.generate_content(prompt_text)

        # Handle potential blocking or empty responses
        if not response.parts:
            try:
                if response.prompt_feedback and response.prompt_feedback.block_reason:
                    return (f"❌ Content blocked by Gemini. "
                            f"Reason: {response.prompt_feedback.block_reason}. "
                            f"Message: {response.prompt_feedback.block_reason_message}")
                elif response.candidates and response.candidates[0].finish_reason != 'STOP':
                    # Handle other finish reasons like SAFETY, RECITATION etc.
                    safety_ratings_str = str(getattr(response.candidates[0], 'safety_ratings', 'N/A'))
                    return (f"❌ Gemini response stopped. "
                            f"Reason: {response.candidates[0].finish_reason}. "
                            f"Safety Ratings: {safety_ratings_str}")
                else:
                    return "❌ Gemini returned an empty response for an unknown reason."
            except (AttributeError, IndexError):
                 return f"❌ Gemini returned an empty or blocked response (unexpected structure). Full response: {response}"
            except Exception as e:
                 return f"❌ Error parsing Gemini blocked response details: {e}. Full response: {response}"

        return response.text
    except ImportError:
        return "❌ Error: 'google-generativeai' library not found. Please ensure Cell 1 ran successfully."
    except google_exceptions.PermissionDenied:
         return f"❌ Google AI Permission Denied: Invalid API key or API not enabled."
    except google_exceptions.ResourceExhausted:
         return f"❌ Google AI Quota Exceeded: You have exceeded your usage limit."
    except Exception as e:
        return f"❌ An unexpected error occurred calling Gemini: {e}"

def call_claude_api(api_key, prompt_text, model="claude-3-haiku-20240307", system_message="You are a helpful assistant specialized in humanitarian negotiation analysis."):
    """
    Makes a real API call to Anthropic (Claude).

    Args:
        api_key (str): Anthropic API key.
        prompt_text (str): The prompt to send to the model.
        model (str): The Claude model to use.
        system_message (str): System message to control assistant behavior.

    Returns:
        str: The response text from the model or an error message.
    """
    if not api_key:
        return "❌ Error: Anthropic API key not provided."
    try:
        from anthropic import Anthropic, AuthenticationError, APIError
        client = Anthropic(api_key=api_key)
        message = client.messages.create(
            model=model,
            max_tokens=4096,  # Increased token limit for complex scenarios
            system=system_message,
            messages=[
                {
                    "role": "user",
                    "content": prompt_text
                }
            ]
        )
        response_text = ""
        if message.content:
            for block in message.content:
                if block.type == "text":
                    response_text += block.text

        if not response_text and message.stop_reason:
            return f"⚠️ Claude response empty. Stop Reason: {message.stop_reason}"
        elif not response_text:
             return "⚠️ Claude returned an empty response for an unknown reason."

        return response_text
    except ImportError:
        return "❌ Error: 'anthropic' library not found. Please ensure Cell 1 ran successfully."
    except AuthenticationError:
         return f"❌ Anthropic Authentication Error: Invalid API key."
    except APIError as e:
         return f"❌ Anthropic API Error: {e}"
    except Exception as e:
        return f"❌ An unexpected error occurred calling Claude: {e}"

def call_mistral_api(api_key, prompt_text, model="mistral-small-latest"):
    """
    Makes a real API call to Mistral AI.

    Args:
        api_key (str): Mistral AI API key.
        prompt_text (str): The prompt to send to the model.
        model (str): The Mistral model to use.

    Returns:
        str: The response text from the model or an error message.
    """
    if not api_key:
        return "❌ Error: Mistral API key not provided."
    try:
        from mistralai.client import MistralClient
        from mistralai.models.chat_completion import ChatMessage
        from mistralai.exceptions import MistralAPIException, MistralConnectionException

        client = MistralClient(api_key=api_key)
        chat_response = client.chat(
            model=model,
            messages=[ChatMessage(role="user", content=prompt_text)]
        )
        return chat_response.choices[0].message.content
    except ImportError:
        return "❌ Error: 'mistralai' library not found. Please ensure Cell 1 ran successfully."
    except MistralAPIException as e:
         # Handle specific Mistral errors, e.g., authentication, rate limits
         return f"❌ Mistral API Error: {e}"
    except MistralConnectionException as e:
         return f"❌ Mistral Connection Error: {e}"
    except Exception as e:
        return f"❌ An unexpected error occurred calling Mistral: {e}"

def call_huggingface_api(api_key, prompt_text, model="mistralai/Mistral-7B-Instruct-v0.2"):
    """
    Makes a real API call to Hugging Face Inference API.

    Args:
        api_key (str): Hugging Face API key.
        prompt_text (str): The prompt to send to the model.
        model (str): The model to use via Hugging Face.

    Returns:
        str: The response text from the model or an error message.
    """
    if not api_key:
        return "❌ Error: Hugging Face API key not provided."

    API_URL = f"https://api-inference.huggingface.co/models/{model}"
    headers = {"Authorization": f"Bearer {api_key}"}

    try:
        payload = {
            "inputs": prompt_text,
            "parameters": {
                "max_new_tokens": 1024,
                "temperature": 0.7,
                "top_p": 0.95,
                "do_sample": True
            }
        }

        response = requests.post(API_URL, headers=headers, json=payload)

        if response.status_code == 200:
            result = response.json()
            # Different models might have different response structures
            if isinstance(result, list) and len(result) > 0:
                if "generated_text" in result[0]:
                    return result[0]["generated_text"]
            elif isinstance(result, dict):
                if "generated_text" in result:
                    return result["generated_text"]

            # Fallback if we can't find the expected structure
            return str(result)
        else:
            return f"❌ Hugging Face API Error: {response.status_code}, {response.text}"

    except Exception as e:
        return f"❌ An unexpected error occurred calling Hugging Face: {e}"

# --- Local LLM API Call Function (REAL - Requires Local Ollama) ---

def call_ollama_api(model_name, prompt_text, ollama_base_url="http://localhost:11434"):
    """
    Makes a real API call to a locally running Ollama instance.

    Args:
        model_name (str): The name of the Ollama model to use (e.g., 'llama3:8b').
        prompt_text (str): The prompt to send to the model.
        ollama_base_url (str): The base URL of the Ollama API endpoint.

    Returns:
        str: The response text from the model or an error message.
    """
    if not model_name:
        return "❌ Error: Ollama model name not provided."
    if not prompt_text:
        return "⚠️ Warning: Prompt is empty."

    api_url = f"{ollama_base_url.rstrip('/')}/api/generate"
    payload = {
        "model": model_name,
        "prompt": prompt_text,
        "stream": False # Get the full response at once
    }
    headers = {'Content-Type': 'application/json'}

    try:
        print(f"ℹ️ Attempting to contact Ollama at {api_url} with model '{model_name}'...")
        response = requests.post(api_url, json=payload, headers=headers, timeout=120) # Increased timeout
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)

        response_data = response.json()
        # Check if the 'response' key exists
        if 'response' in response_data:
            return response_data['response'].strip()
        elif 'error' in response_data:
             # Handle cases where Ollama itself returns an error (e.g., model not found)
             return f"❌ Ollama API Error: {response_data['error']}"
        else:
             # Handle unexpected response structure
             return f"❌ Ollama Error: Unexpected response format. Full response: {response_data}"

    except requests.exceptions.ConnectionError:
        return (f"❌ Connection Error: Could not connect to Ollama at {ollama_base_url}. "
                f"Ensure Ollama is installed, running, and accessible from where you run this script. "
                f"If running Gradio in Colab, you might need tunneling (e.g., cloudflared) or run Gradio locally.")
    except requests.exceptions.Timeout:
        return f"❌ Timeout Error: The request to Ollama at {api_url} timed out."
    except requests.exceptions.RequestException as e:
        # Catch other potential request errors
        return f"❌ Request Error: An error occurred contacting Ollama: {e}"
    except json.JSONDecodeError:
        return f"❌ Ollama Error: Could not decode JSON response from Ollama. Response text: {response.text}"
    except Exception as e:
        # Catch any other unexpected errors
        return f"❌ An unexpected error occurred calling Ollama: {e}"

# Alternative method using subprocess if Ollama API approach doesn't work
def call_ollama_subprocess(model_name, prompt_text):
    """
    Uses subprocess to call Ollama CLI directly (alternative method).

    Args:
        model_name (str): Name of the Ollama model (e.g., "gemma:2b")
        prompt_text (str): Prompt text to send to the model

    Returns:
        str: Model response or error message
    """
    try:
        # Escape the prompt for shell
        import shlex
        escaped_prompt = shlex.quote(prompt_text)

        # Run the Ollama command
        result = subprocess.run(
            f"ollama run {model_name} {escaped_prompt}",
            shell=True,
            capture_output=True,
            text=True,
            timeout=120  # 2 minute timeout
        )

        if result.returncode == 0:
            return result.stdout.strip()
        else:
            return f"❌ Ollama Error: {result.stderr}"
    except subprocess.TimeoutExpired:
        return "❌ Ollama subprocess timed out (2 minutes)"
    except Exception as e:
        return f"❌ Error running Ollama via subprocess: {e}"

# --- n8n Webhook Trigger Function (REAL) ---

def trigger_n8n_webhook(webhook_url, payload_data):
    """
    Sends data to a specified n8n webhook URL.

    Args:
        webhook_url (str): The full URL of the n8n webhook.
        payload_data (str): A string containing the JSON payload to send.

    Returns:
        str: A status message indicating success or failure.
    """
    if not webhook_url:
        return "❌ Error: n8n Webhook URL not provided."
    if not payload_data:
        return "⚠️ Warning: Payload is empty."

    headers = {'Content-Type': 'application/json'}
    try:
        # Validate and parse the JSON payload string
        payload_dict = json.loads(payload_data)
    except json.JSONDecodeError as e:
        return f"❌ JSON Error: Invalid JSON format in payload data. Details: {e}"

    try:
        print(f"ℹ️ Sending data to n8n webhook: {webhook_url}")
        response = requests.post(webhook_url, json=payload_dict, headers=headers, timeout=30)
        response.raise_for_status() # Raise HTTPError for bad responses (4xx or 5xx)
        # Try to parse n8n's response (often JSON)
        try:
            n8n_response = response.json()
            return f"✅ Successfully triggered n8n workflow.\nResponse: {json.dumps(n8n_response, indent=2)}"
        except json.JSONDecodeError:
            # If response is not JSON, return the text
            return f"✅ Successfully triggered n8n workflow.\nResponse: {response.text}"

    except requests.exceptions.ConnectionError:
        return (f"❌ Connection Error: Could not connect to the n8n webhook URL: {webhook_url}. "
                f"Ensure the URL is correct and the n8n instance/workflow is running and accessible.")
    except requests.exceptions.Timeout:
        return f"❌ Timeout Error: The request to the n8n webhook timed out."
    except requests.exceptions.HTTPError as e:
         # Handle HTTP errors (like 404 Not Found, 401 Unauthorized, 500 Internal Server Error)
         return (f"❌ HTTP Error: Failed to trigger n8n webhook. Status Code: {e.response.status_code}. "
                 f"Response: {e.response.text}")
    except requests.exceptions.RequestException as e:
        return f"❌ Request Error: An error occurred sending data to n8n: {e}"
    except Exception as e:
        return f"❌ An unexpected error occurred triggering n8n: {e}"


# --- ElevenLabs Text-to-Speech Function (REAL) ---

def generate_elevenlabs_audio(api_key, text_to_speak, voice_id="Rachel", model_id='eleven_multilingual_v2',
                              stability=0.7, similarity_boost=0.5):
    """
    Generates audio using the real ElevenLabs API with enhanced parameters.

    Args:
        api_key (str): ElevenLabs API key.
        text_to_speak (str): The text to convert to speech.
        voice_id (str): The ID or name of the ElevenLabs voice to use.
        model_id (str): The ElevenLabs model to use.
        stability (float): Voice stability setting (0.0-1.0).
        similarity_boost (float): Voice similarity boost setting (0.0-1.0).

    Returns:
        tuple: (filepath_or_none, status_message)
               filepath_or_none (str | None): Path to the generated MP3 file or None on error.
               status_message (str): A message indicating success or failure.
    """
    if not api_key:
        return None, "❌ Error: ElevenLabs API key not provided."
    if not text_to_speak:
        return None, "⚠️ Warning: No text provided for synthesis."

    # Create a directory for audio output if it doesn't exist
    output_dir = Path("./elevenlabs_output")
    output_dir.mkdir(parents=True, exist_ok=True)

    try:
        from elevenlabs.client import ElevenLabs
        from elevenlabs import Voice, VoiceSettings, APIError
        client = ElevenLabs(api_key=api_key)

        print(f"ℹ️ ElevenLabs: Generating audio for '{text_to_speak[:50]}...' with voice '{voice_id}'")
        audio_stream = client.generate(
            text=text_to_speak,
            voice=Voice(
                voice_id=voice_id, # Can be voice ID or name
                settings=VoiceSettings(stability=stability, similarity_boost=similarity_boost)
            ),
            model=model_id
        )

        # Save audio stream to a temporary file
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_filename = output_dir / f"audio_{timestamp}.mp3"

        with open(output_filename, "wb") as f:
            data_written = False
            for chunk in audio_stream:
                if chunk:
                    f.write(chunk)
                    data_written = True

        if data_written and output_filename.exists() and output_filename.stat().st_size > 0:
            print(f"✅ Audio saved as {output_filename}")
            # Return the absolute path as a string for Gradio
            return str(output_filename.resolve()), f"✅ Audio generated successfully with voice '{voice_id}'."
        else:
            # Clean up empty file if created
            if output_filename.exists():
                output_filename.unlink()
            return None, f"⚠️ Warning: ElevenLabs did not return audio data for the provided text/voice."

    except ImportError:
         return None, "❌ Error: 'elevenlabs' library not found. Please ensure Cell 1 ran successfully."
    except APIError as e:
         # Handle specific ElevenLabs API errors
         if e.status_code == 401:
             return None, f"❌ ElevenLabs Authentication Error: Invalid API key. ({e})"
         elif "quota" in str(e).lower() or e.status_code == 402:
              return None, f"❌ ElevenLabs Quota Exceeded: Check your subscription limits. ({e})"
         else:
             return None, f"❌ ElevenLabs API Error: {e}"
    except Exception as e:
        # Catch other unexpected errors
        return None, f"❌ An unexpected error occurred calling ElevenLabs: {e}"

# Local text-to-speech fallback
def generate_local_tts(text, output_dir="./tts_output"):
    """
    Generate text-to-speech using local pyttsx3 library (fallback when no API key).

    Args:
        text (str): Text to convert to speech
        output_dir (str): Directory to save the audio file

    Returns:
        tuple: (filepath_or_none, status_message)
    """
    if not text:
        return None, "⚠️ Warning: No text provided for synthesis."

    try:
        # Create output directory if it doesn't exist
        output_dir = Path(output_dir)
        output_dir.mkdir(parents=True, exist_ok=True)

        # Initialize TTS engine
        engine = pyttsx3.init()

        # Generate filename
        timestamp = datetime.now().strftime("%Y%m%d_%H%M%S")
        output_file = output_dir / f"local_tts_{timestamp}.mp3"

        # Save to file
        engine.save_to_file(text, str(output_file))
        engine.runAndWait()

        if output_file.exists() and output_file.stat().st_size > 0:
            return str(output_file.resolve()), "✅ Audio generated successfully using local TTS."
        else:
            return None, "❌ Failed to generate audio file with local TTS."

    except Exception as e:
        return None, f"❌ Error with local TTS: {e}"

# --- Helper Function for Negotiation Frameworks ---

def apply_negotiation_framework(framework_name, scenario_text):
    """
    Creates a prompt asking an LLM to apply a specific negotiation framework.

    Args:
        framework_name (str): Name of the framework ('Island of Agreement', 'BATNA/ZOPA', etc.).
        scenario_text (str): The negotiation scenario text.

    Returns:
        str: A formatted prompt for the LLM.
    """
    if not framework_name or framework_name.lower() == 'none':
        return scenario_text # Return original text if no framework selected

    prompt_header = f"Analyze the following humanitarian negotiation scenario using the '{framework_name}' framework:\n\nScenario:\n{scenario_text}\n\nAnalysis ({framework_name}):\n"

    if framework_name == 'Island of Agreement':
        prompt_details = ("Identify and structure the analysis into these four categories:\n"
                          "1. Contested Facts/Issues: Points of disagreement or differing information.\n"
                          "2. Agreed Facts/Issues: Points where both parties have common ground or agreement.\n"
                          "3. Divergent Norms/Values/Interests: Underlying principles or goals that differ.\n"
                          "4. Convergent Norms/Values/Interests: Shared principles or goals that can be built upon.")
    elif framework_name == 'BATNA/ZOPA':
        prompt_details = ("Identify the following for EACH key party involved (e.g., Humanitarian Org, Armed Group Alpha):\n"
                          "1. Interests: What are their underlying needs and motivations?\n"
                          "2. Options: What are possible solutions or agreements?\n"
                          "3. BATNA (Best Alternative To a Negotiated Agreement): What will they do if no agreement is reached?\n"
                          "4. ZOPA (Zone Of Possible Agreement): Is there an overlap where an agreement beneficial to both might exist? Describe it if so.")
    elif framework_name == 'Iceberg Analysis':
        prompt_details = ("For each key party in the negotiation scenario:\n"
                         "1. Position: What they say they want (visible part of iceberg)\n"
                         "2. Reasoning: Why they want it (just below surface)\n"
                         "3. Motives: Deeper needs and fears (deep underwater)\n\n"
                         "Then provide strategic insights on finding common ground based on underlying interests.")
    elif framework_name == 'Stakeholder Matrix':
        prompt_details = ("For each stakeholder in the scenario:\n"
                         "1. Power: High/Medium/Low - Their ability to influence outcomes\n"
                         "2. Legitimacy: High/Medium/Low - Their recognized right to be involved\n"
                         "3. Urgency: High/Medium/Low - How time-sensitive their claims are\n"
                         "4. Position: Their stance on the key issues\n"
                         "5. Priority: Critical/High/Medium/Low - Overall engagement priority\n\n"
                         "Then provide strategic insights on which stakeholders to prioritize.")
    elif framework_name == 'Multi-party Dynamics':
        prompt_details = ("Analyze the complex relationships between multiple parties:\n"
                         "1. Identify potential coalitions and their shared concerns\n"
                         "2. Map relationships between different stakeholder groups\n"
                         "3. Develop strategies for coalition-building\n"
                         "4. Identify potential spoilers and isolation strategies")
    elif framework_name == 'Success Factors':
        prompt_details = ("Identify critical elements needed for resolution:\n"
                         "1. Resolution Path: The overall approach to successful resolution\n"
                         "2. Trust-Building Actions: Steps to build confidence between parties\n"
                         "3. Stakeholder Management: How to engage different actors effectively\n"
                         "4. Process Enhancement: Improvements to negotiation structure and methodology")
    elif framework_name == 'Complete Crisis Framework':
        prompt_details = ("Create a comprehensive analysis using the Enhanced Crisis Negotiation Analyst framework with ALL of these components:\n"
                          "1. Island of Agreements: Contested facts, agreed facts, convergent norms, divergent norms\n"
                          "2. Iceberg Analysis: Underlying interests beneath stated positions for each party\n"
                          "3. Stakeholder Matrix: Map key actors by power, legitimacy, and urgency\n"
                          "4. Influence Pathways: Relationships and influence channels between stakeholders\n"
                          "5. Scenario Design: Negotiation boundaries and potential agreement zones\n"
                          "6. Multiparty Dynamics: Complex relationships and coalition opportunities\n"
                          "7. Tactical Timeline: Structured approach to resolution through phases\n"
                          "8. Radical Faction Analysis: Identify disruptive elements and management strategies\n"
                          "9. Success Factors: Critical elements for resolution\n\n"
                          "Provide a thorough analysis covering all components with actionable recommendations.")
    else:
        prompt_details = "Provide a detailed analysis based on the principles of this framework." # Generic fallback

    return f"{prompt_header}\n{prompt_details}"

# --- Helper Function for Integrated Assistant ---

def build_assistant_prompt(scenario_text):
    """
    Creates a structured prompt for the Integrated Assistant LLM call.

    Args:
        scenario_text (str): The negotiation scenario text.

    Returns:
        str: A formatted prompt asking for structured analysis.
    """
    return f"""
    Analyze the following humanitarian negotiation scenario comprehensively. Provide the output structured into the following sections:

    **1. Executive Summary:**
    (Provide a brief overview of the situation and key negotiation challenges/opportunities in 2-3 sentences.)

    **2. Stakeholder Analysis:**
    (Identify key stakeholders (e.g., Humanitarian Org, Armed Group Alpha, IDP Leaders, Local Authorities). For each, list:
    * Stated Position: What they say they want.
    * Underlying Interests: Why they want it (needs, fears, motivations).
    * Potential Influence: How much power do they have in this situation?
    * Possible Levers: What might influence them?)

    **3. Contextual Factors:**
    (Briefly describe relevant factors like: security situation, political climate, cultural norms, past interactions, resource availability.)

    **4. Negotiation Framework Analysis (IoA & BATNA/ZOPA):**
    * **Island of Agreement:** Briefly list key Contested Facts, Agreed Facts, Divergent Norms/Interests, Convergent Norms/Interests.
    * **BATNA/ZOPA:** Briefly outline the likely BATNA for the main parties and assess if a ZOPA exists.

    **5. Key Risks & Challenges:**
    (List the main risks for the humanitarian organization, e.g., security, reputational, operational failure, ethical dilemmas.)

    **6. Strategic Recommendations:**
    (Provide 3-5 concrete, actionable recommendations for the negotiation team, focusing on initial steps and potential approaches. Consider sequencing, communication strategies, and potential entry points.)

    **Scenario:**
    {scenario_text}

    **Analysis:**
    """

# --- NEW: Function to build Complete Analysis Framework Prompt ---

def build_crisis_analysis_prompt(scenario_text, output_format="text"):
    """
    Creates a structured prompt for the Crisis Negotiation Analysis Framework.

    Args:
        scenario_text (str): The crisis negotiation scenario text.
        output_format (str): Output format - "text" or "json"

    Returns:
        str: A formatted prompt asking for structured analysis.
    """
    json_instructions = """
    Your output must be valid JSON that precisely follows this structure:
    {
        "title": "🏛️ [Scenario Name] Crisis Negotiation Analysis 🌉",
        "subtitle": "💢 [Brief description of situation] 🔄",
        "description": "📊 Strategic analysis of [scenario context] with recommendations for resolution 🧩",
        "executiveSummary": "📝 [1-2 paragraph overview] 🔎",
        "strategicApproach": "🎯 [Multi-track strategy explained] 📈",
        "situationDescription": "⚠️ [Detailed situation description] 🏢",
        "toolsOverview": "🧰 [Brief explanation of frameworks used] 📈",
        "islandOfAgreements": {
            "description": "🏝️ [Framework explanation] 🗣️",
            "strategy": "🔍 [Strategic approach to use framework] 📚",
            "contestedFacts": [
                "⚖️ [Contested fact 1]",
                "🛑 [Contested fact 2]",
                "🔥 [Contested fact 3]"
            ],
            "agreedFacts": [
                "💲 [Agreed fact 1]",
                "🏠 [Agreed fact 2]",
                "✊ [Agreed fact 3]"
            ],
            "convergentNorms": [
                "🛡️ [Convergent norm 1]",
                "❌ [Convergent norm 2]",
                "🎓 [Convergent norm 3]"
            ],
            "divergentNorms": [
                "⚔️ [Divergent norm 1]",
                "🚫 [Divergent norm 2]",
                "💰 [Divergent norm 3]"
            ],
            "factualPath": [
                "📋 [Factual path element 1]",
                "📝 [Factual path element 2]"
            ],
            "normativePath": [
                "✅ [Normative path element 1]",
                "🙏 [Normative path element 2]"
            ]
        },
        "icebergAnalysis": {
            "description": "❄️ [Framework explanation] 🧊",
            "party1": {
                "name": "🏛️ [Party 1 name]",
                "position": "⛔ [Stated position]",
                "reasoning": "🏢 [Reasoning]",
                "motives": "🎓 [Underlying motives]"
            },
            "party2": {
                "name": "👥 [Party 2 name]",
                "position": "✊ [Stated position]",
                "reasoning": "⚖️ [Reasoning]",
                "motives": "🔑 [Underlying motives]"
            },
            "insight": "🔍 [Key insight from analysis] 🌉",
            "strategies": [
                {
                    "title": "🔄 [Strategy 1 name]",
                    "description": "🗣️ [Strategy 1 description]"
                },
                {
                    "title": "🤝 [Strategy 2 name]",
                    "description": "👂 [Strategy 2 description]"
                }
            ]
        },
        "stakeholderMatrix": {
            "description": "👥 [Framework explanation] ⚖️",
            "insight": "🎯 [Key insight from analysis] 🌉",
            "stakeholders": [
                {
                    "name": "🏛️ [Stakeholder 1]",
                    "power": "High",
                    "legitimacy": "High",
                    "urgency": "High",
                    "position": "🎯 [Position]",
                    "priority": "Critical"
                },
                {
                    "name": "👨‍👩‍👧‍👦 [Stakeholder 2]",
                    "power": "Medium",
                    "legitimacy": "High",
                    "urgency": "Medium",
                    "position": "🎯 [Position]",
                    "priority": "High"
                }
            ]
        },
        "influencePathways": {
            "description": "🕸️ [Framework explanation] 🔀",
            "strategy": "🎯 [Strategic approach] 📱",
            "pathways": [
                "🔄 [Pathway 1]",
                "🤝 [Pathway 2]"
            ],
            "quadrantStrategies": {
                "alliance": [
                    "🔄 [Alliance strategy 1]",
                    "📊 [Alliance strategy 2]"
                ],
                "coalition": [
                    "🌉 [Coalition strategy 1]",
                    "🤝 [Coalition strategy 2]"
                ],
                "cooperation": [
                    "🙏 [Cooperation strategy 1]",
                    "🛠️ [Cooperation strategy 2]"
                ],
                "mitigation": [
                    "🗣️ [Mitigation strategy 1]",
                    "👮 [Mitigation strategy 2]"
                ]
            },
            "nodes": [
                {
                    "id": "admin",
                    "label": "🏛️ [Node 1]",
                    "type": "core",
                    "x": 0,
                    "y": 0,
                    "color": "blue"
                },
                {
                    "id": "board",
                    "label": "👑 [Node 2]",
                    "type": "quadrant1",
                    "x": -25,
                    "y": -25,
                    "color": "purple"
                }
            ],
            "connections": [
                {
                    "source": "admin",
                    "target": "board",
                    "label": "🔄 [Connection 1]"
                },
                {
                    "source": "admin",
                    "target": "security",
                    "label": "👮 [Connection 2]"
                }
            ]
        },
        "scenarioDesign": {
            "description": "🧩 [Framework explanation] 🎯",
            "insight": "🔍 [Key insight] 🎯",
            "party1": {
                "name": "🏛️ [Party 1]",
                "ideal": "⚡ [Ideal outcome]",
                "bottomLine": "🤝 [Bottom line]",
                "redLine": "❌ [Red line]"
            },
            "party2": {
                "name": "👥 [Party 2]",
                "ideal": "⚡ [Ideal outcome]",
                "bottomLine": "🤝 [Bottom line]",
                "redLine": "❌ [Red line]"
            },
            "areaF": [
                "🏠 [Area F element 1]",
                "⚖️ [Area F element 2]"
            ],
            "areaE": [
                "👥 [Area E element 1]",
                "🛡️ [Area E element 2]"
            ],
            "areaD": [
                "🚫 [Area D element 1]",
                "👥 [Area D element 2]"
            ],
            "recommendations": [
                "⚙️ [Recommendation 1]",
                "📅 [Recommendation 2]"
            ]
        },
        "multipartyDynamics": {
            "description": "👥 [Framework explanation] 🤝",
            "insight": "🔍 [Key insight] 🌉",
            "coalitions": [
                {
                    "name": "🏛️ [Coalition 1]",
                    "concern": "🎯 [Main concern]",
                    "members": [
                        "🎓 [Member 1]",
                        "👑 [Member 2]"
                    ]
                },
                {
                    "name": "✊ [Coalition 2]",
                    "concern": "🎯 [Main concern]",
                    "members": [
                        "👥 [Member 1]",
                        "🗣️ [Member 2]"
                    ]
                }
            ],
            "strategies": [
                {
                    "title": "🌉 [Strategy 1]",
                    "description": "👨‍🏫 [Description]"
                },
                {
                    "title": "⚔️ [Strategy 2]",
                    "description": "🧩 [Description]"
                }
            ]
        },
        "tacticalTimeline": {
            "description": "⏱️ [Framework explanation] 📅",
            "currentStatus": "🔄 [Current situation] 🔍",
            "criticalPath": [
                "🛡️ [Critical path element 1]",
                "🎯 [Critical path element 2]"
            ],
            "phases": [
                {
                    "title": "📋 [Phase 1]",
                    "duration": "⏱️ [Duration]",
                    "tasks": [
                        {
                            "text": "🤝 [Task 1]",
                            "status": "pending"
                        },
                        {
                            "text": "👥 [Task 2]",
                            "status": "pending"
                        }
                    ]
                },
                {
                    "title": "🗣️ [Phase 2]",
                    "duration": "⏱️ [Duration]",
                    "tasks": [
                        {
                            "text": "📝 [Task 1]",
                            "status": "pending"
                        },
                        {
                            "text": "📏 [Task 2]",
                            "status": "pending"
                        }
                    ]
                }
            ]
        },
        "radicalFaction": {
            "description": "🔥 [Framework explanation] ⚠️",
            "insight": "🔍 [Key insight] 🛡️",
            "characterization": [
                {
                    "label": "Composition 👥",
                    "value": "📊 [Description]"
                },
                {
                    "label": "Tactics 👑",
                    "value": "🧠 [Description]"
                }
            ],
            "managementStrategies": [
                {
                    "label": "Stakeholder Segmentation 🧩",
                    "value": "⚔️ [Description]"
                },
                {
                    "label": "Communication Strategy 🔍",
                    "value": "⚖️ [Description]"
                }
            ],
            "risks": [
                {
                    "action": "🔨 [Risk 1]",
                    "likelihood": "High",
                    "impact": "Critical",
                    "mitigation": "🔎 [Mitigation approach]"
                },
                {
                    "action": "👊 [Risk 2]",
                    "likelihood": "Medium",
                    "impact": "High",
                    "mitigation": "🛡️ [Mitigation approach]"
                }
            ]
        },
        "successFactors": {
            "description": "🎯 [Framework explanation] ⭐",
            "resolutionPath": "🛣️ [Overall resolution approach] 🏆",
            "factors": [
                {
                    "name": "🤝 [Factor 1]",
                    "description": "⚖️ [Description]",
                    "progress": 15
                },
                {
                    "name": "⚙️ [Factor 2]",
                    "description": "🧩 [Description]",
                    "progress": 10
                }
            ],
            "recommendations": {
                "trustBuilding": [
                    "🛠️ [Trust recommendation 1]",
                    "🗣️ [Trust recommendation 2]"
                ],
                "stakeholderManagement": [
                    "📝 [Stakeholder recommendation 1]",
                    "👨‍🏫 [Stakeholder recommendation 2]"
                ],
                "processEnhancement": [
                    "⏱️ [Process recommendation 1]",
                    "📋 [Process recommendation 2]"
                ]
            }
        },
        "powerRelationships": [
            "🏛️ [Power relationship 1]",
            "👥 [Power relationship 2]"
        ],
        "islandInsight": "🏝️ [Island framework insight]",
        "icebergInsight": "❄️ [Iceberg framework insight]",
        "stakeholderInsight": "👥 [Stakeholder framework insight]",
        "influenceInsight": "🕸️ [Influence framework insight]",
        "scenarioInsight": "🧩 [Scenario framework insight]",
        "multipartyInsight": "🤝 [Multiparty framework insight]",
        "tacticalInsight": "⏱️ [Tactical framework insight]",
        "radicalInsight": "🔥 [Radical framework insight]",
        "successInsight": "🎯 [Success framework insight]"
    }
    """

    base_prompt = """
    You are a Crisis Negotiation Analyst specialized in producing thorough analytical breakdowns of complex humanitarian negotiation scenarios.

    Analyze the following scenario comprehensively using all 9 specialized analytical frameworks:

    1. Island of Agreements (IoA) Framework - Identify starting points for dialogue
    2. Iceberg Analysis - Reveal underlying interests beneath stated positions
    3. Stakeholder Matrix - Map key actors based on power, legitimacy, and urgency
    4. Influence Pathways - Visualize key relationships and influence channels
    5. Scenario Design - Map negotiation position boundaries and identify potential agreement zones
    6. Multiparty Dynamics - Map complex stakeholder relationships and coalition-building opportunities
    7. Tactical Timeline - Structured approach to resolution through sequential phases
    8. Radical Faction Analysis - Identify potentially disruptive elements and risk management strategies
    9. Success Factors - Define critical elements needed for resolution

    For each framework, provide detailed analysis with specific recommendations.

    Scenario:
    {scenario}
    """

    if output_format == "json":
        return base_prompt.format(scenario=scenario_text) + "\n\n" + json_instructions
    else:
        return base_prompt.format(scenario=scenario_text)

# --- NEW: Function to handle MTPs (Mini Task Processes) ---

def load_mtps_from_json(filepath="mtp_examples.json"):
    """
    Load Mini Task Processes from a JSON file.

    Args:
        filepath (str): Path to JSON file

    Returns:
        list: List of MTP dictionaries or empty list on error
    """
    try:
        with open(filepath, 'r') as f:
            mtps = json.load(f)
        return mtps
    except FileNotFoundError:
        print(f"❌ MTP file not found: {filepath}")
        # Create default MTPs if file not found
        return create_default_mtps()
    except json.JSONDecodeError:
        print(f"❌ Invalid JSON in MTP file: {filepath}")
        return create_default_mtps()
    except Exception as e:
        print(f"❌ Error loading MTPs: {e}")
        return create_default_mtps()

def create_default_mtps():
    """
    Create default MTPs for first-time use.

    Returns:
        list: List of default MTP dictionaries
    """
    default_mtps = [
        {
            "title": "Island of Agreement Analysis",
            "description": "Identifies potential starting points for dialogue by mapping areas of convergence and divergence",
            "prompt_template": "Analyze the following humanitarian negotiation scenario using the Island of Agreement framework: {input}\n\nIdentify and categorize:\n1. Contested Facts/Issues\n2. Agreed Facts/Issues\n3. Divergent Norms/Values/Interests\n4. Convergent Norms/Values/Interests",
            "preferred_model": "Claude (Anthropic)",
            "send_to_n8n": False,
            "speak_output": False
        },
        {
            "title": "Stakeholder Mapping",
            "description": "Maps key actors based on power, legitimacy, and urgency",
            "prompt_template": "Create a comprehensive stakeholder analysis for this humanitarian negotiation scenario: {input}\n\nFor each key stakeholder, identify:\n1. Power level (High/Medium/Low)\n2. Legitimacy (High/Medium/Low)\n3. Urgency (High/Medium/Low)\n4. Position on key issues\n5. Priority for engagement (Critical/High/Medium/Low)",
            "preferred_model": "GPT-4 (OpenAI)",
            "send_to_n8n": True,
            "speak_output": False
        },
        {
            "title": "Critical Security Briefing",
            "description": "Creates a concise security briefing for field staff with actionable points",
            "prompt_template": "Create a concise security briefing for humanitarian field staff based on this scenario: {input}\n\nInclude:\n1. Key security threats (ranked by severity)\n2. Recommended precautions\n3. Emergency protocols\n4. Communication procedures\n\nFormat this as a brief, actionable document that could be read aloud in 2-3 minutes.",
            "preferred_model": "GPT-4 (OpenAI)",
            "send_to_n8n": True,
            "speak_output": True
        },
        {
            "title": "Tactical Timeline",
            "description": "Develops a phased approach to negotiations with specific milestones",
            "prompt_template": "Create a tactical timeline for resolving this humanitarian negotiation scenario: {input}\n\nStructure this into:\n1. Current Status (assessment of where we are now)\n2. Critical Path (key steps needed)\n3. Phase 1: Initial Engagement (2-3 days)\n4. Phase 2: Core Negotiation (4-7 days)\n5. Phase 3: Implementation (7+ days)\n\nFor each phase, identify specific tasks, responsible parties, and success criteria.",
            "preferred_model": "Claude (Anthropic)",
            "send_to_n8n": False,
            "speak_output": False
        },
        {
            "title": "Complete Crisis Analysis",
            "description": "Comprehensive analysis using all 9 frameworks with JSON output",
            "prompt_template": "Perform a complete crisis negotiation analysis on this scenario, using all 9 analytical frameworks (Island of Agreement, Iceberg Analysis, Stakeholder Matrix, Influence Pathways, Scenario Design, Multiparty Dynamics, Tactical Timeline, Radical Faction Analysis, Success Factors). Format the output as JSON with each framework as a separate section: {input}",
            "preferred_model": "Claude (Anthropic)",
            "send_to_n8n": True,
            "speak_output": False
        }
    ]

    # Try to save default MTPs for future use
    try:
        with open("mtp_examples.json", 'w') as f:
            json.dump(default_mtps, f, indent=2)
        print("✅ Created default MTP file")
    except Exception as e:
        print(f"⚠️ Could not save default MTPs: {e}")

    return default_mtps

def save_new_mtp(mtp_data, filepath="mtp_examples.json"):
    """
    Save a new MTP to the JSON file.

    Args:
        mtp_data (dict): New MTP to save
        filepath (str): Path to JSON file

    Returns:
        bool: Success status
    """
    try:
        # Load existing MTPs
        mtps = load_mtps_from_json(filepath)

        # Add new MTP
        mtps.append(mtp_data)

        # Save back to file
        with open(filepath, 'w') as f:
            json.dump(mtps, f, indent=2)

        return True
    except Exception as e:
        print(f"❌ Error saving new MTP: {e}")
        return False

print("✅ Utility functions and API callers defined.")


# @title Cell 3: Gradio Interface Implementation
# ---
# Defines the Gradio application structure using Blocks and Tabs.
# Connects interface components to the utility functions from Cell 2.
# All UI text is in English.
# ---
import gradio as gr
import os
import json
import matplotlib.pyplot as plt
import pandas as pd
import base64 # For embedding HTML interface assets
import time # For timing API responses

print("⏳ Creating Gradio interface...")

# --- Case Study Text (English) ---
CASE_STUDY_EN = """
**Case Study: Negotiation for Humanitarian Access**

An armed conflict in the North Province has resulted in approximately 20,000 internally displaced persons (IDPs) gathering in a makeshift camp near the town of Eastville. Armed Group Alpha controls the area surrounding the camp and has established checkpoints on all access roads. Your humanitarian organization needs to negotiate access to provide essential services including food, water, medical care, and protection services.
"""

# --- Edward University Case Study (from the JSON) ---
EDWARD_UNIVERSITY_CASE = """
**Case Study: Edward University Campus Crisis**

A peaceful student protest over tuition fees, housing conditions, and campus safety has escalated into an occupation of the administration building at Edward University, complicated by the arrest of a dozen students, the infiltration of external activists with extremist views, and intense media coverage politicizing the event. Different student factions are now in conflict over approach, with tensions at a breaking point as the situation devolves into confusion and violence.

The university administration is seeking to end the occupation immediately and restore normal campus operations, while the student union's moderate faction wants to continue protest until concrete commitments address tuition, housing, and safety concerns. External activist groups have joined the protests and introduced divisive elements, some with extremist agendas.

Local police maintain public order but face political considerations and campus jurisdiction issues. Faculty members have limited formal authority but significant potential as mediators with credibility on all sides. Media coverage is amplifying tensions by criticizing government response and politicizing events.
"""

# --- Island of Agreement Interface Code ---
ioa_interface_html = """
<!DOCTYPE html>
<html lang="en">
<head>
    <meta charset="UTF-8">
    <meta name="viewport" content="width=device-width, initial-scale=1.0">
    <title>Island of Agreement Tool</title>
    <link href="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/css/bootstrap.min.css" rel="stylesheet">
    <style>
        body { font-family: sans-serif; padding: 20px; background-color: #f8f9fa; }
        .negotiation-framework { display: grid; grid-template-columns: repeat(auto-fit, minmax(250px, 1fr)); gap: 20px; }
        .framework-section { background-color: white; border-radius: 8px; padding: 15px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); }
        .framework-section h5 { margin-bottom: 15px; color: #0d6efd; border-bottom: 2px solid #dee2e6; padding-bottom: 5px; }
        .item { background-color: #e9ecef; border-radius: 4px; padding: 8px 12px; margin-bottom: 8px; cursor: pointer; transition: background-color 0.2s ease; }
        .item:hover { background-color: #ced4da; }
        .add-item-input { width: calc(100% - 45px); margin-right: 5px; }
        .add-item-btn { width: 40px; }
        #move-options { position: absolute; background-color: white; border: 1px solid #ccc; box-shadow: 0 2px 5px rgba(0,0,0,0.2); padding: 5px; display: none; z-index: 1000; }
        #move-options button { display: block; width: 100%; text-align: left; padding: 5px 10px; border: none; background: none; }
        #move-options button:hover { background-color: #f0f0f0; }
        .context-section { background-color: #fff; padding: 20px; border-radius: 8px; box-shadow: 0 2px 4px rgba(0,0,0,0.1); margin-bottom: 20px; }
        .context-section h4 { color: #6f42c1; } /* Violet color for context */
    </style>
</head>
<body>
    <div class="container-fluid">
        <div class="context-section" id="context-section">
             <h4>Negotiation Context</h4>
             <textarea id="context-input" class="form-control" rows="3" placeholder="Briefly describe the negotiation context or paste relevant text..."></textarea>
             </div>

        <div class="negotiation-framework">
            <div class="framework-section" id="contested-facts">
                <h5>🔴 Contested Facts / Issues</h5>
                <div id="contested-facts-items"></div>
                <div class="input-group input-group-sm mt-2">
                    <input type="text" class="form-control add-item-input" placeholder="Add contested fact...">
                    <button class="btn btn-outline-danger add-item-btn" onclick="addItem('contested-facts', this)">+</button>
                </div>
            </div>
            <div class="framework-section" id="agreed-facts">
                <h5>🟢 Agreed Facts / Issues</h5>
                <div id="agreed-facts-items"></div>
                 <div class="input-group input-group-sm mt-2">
                    <input type="text" class="form-control add-item-input" placeholder="Add agreed fact...">
                    <button class="btn btn-outline-success add-item-btn" onclick="addItem('agreed-facts', this)">+</button>
                </div>
            </div>
            <div class="framework-section" id="divergent-norms">
                <h5>🟠 Divergent Norms / Interests</h5>
                 <div id="divergent-norms-items"></div>
                 <div class="input-group input-group-sm mt-2">
                    <input type="text" class="form-control add-item-input" placeholder="Add divergent norm...">
                    <button class="btn btn-outline-warning add-item-btn" onclick="addItem('divergent-norms', this)">+</button>
                </div>
            </div>
            <div class="framework-section" id="convergent-norms">
                <h5>🔵 Convergent Norms / Interests</h5>
                 <div id="convergent-norms-items"></div>
                 <div class="input-group input-group-sm mt-2">
                    <input type="text" class="form-control add-item-input" placeholder="Add convergent norm...">
                    <button class="btn btn-outline-primary add-item-btn" onclick="addItem('convergent-norms', this)">+</button>
                </div>
            </div>
        </div>
    </div>

    <div id="move-options"></div>

    <script src="https://cdn.jsdelivr.net/npm/bootstrap@5.3.0/dist/js/bootstrap.bundle.min.js"></script>
    <script>
        let currentItem = null;
        const moveOptionsDiv = document.getElementById('move-options');
        const sections = ['contested-facts', 'agreed-facts', 'divergent-norms', 'convergent-norms'];
        const sectionTitles = {
            'contested-facts': '🔴 Contested Facts',
            'agreed-facts': '🟢 Agreed Facts',
            'divergent-norms': '🟠 Divergent Norms',
            'convergent-norms': '🔵 Convergent Norms'
        };

        function addItem(sectionId, buttonElement) {
            const inputElement = buttonElement.previousElementSibling;
            const text = inputElement.value.trim();
            if (text === '') return;

            const itemsContainer = document.getElementById(`${sectionId}-items`);
            const itemDiv = document.createElement('div');
            itemDiv.className = 'item';
            itemDiv.textContent = text;
            itemDiv.onclick = function() {
                showMoveOptions(this, sectionId);
            };
            itemsContainer.appendChild(itemDiv);
            inputElement.value = ''; // Clear input
        }

        function showMoveOptions(itemElement, currentSectionId) {
            currentItem = itemElement;
            moveOptionsDiv.innerHTML = ''; // Clear previous options

            sections.forEach(sectionId => {
                if (sectionId !== currentSectionId) {
                    const button = document.createElement('button');
                    button.textContent = `Move to ${sectionTitles[sectionId]}`;
                    button.onclick = function() {
                        moveItem(sectionId);
                    };
                    moveOptionsDiv.appendChild(button);
                }
            });

            const deleteButton = document.createElement('button');
            deleteButton.textContent = 'Delete Item';
            deleteButton.style.color = 'red';
            deleteButton.onclick = deleteItem;
            moveOptionsDiv.appendChild(deleteButton);

            // Position the move options div near the clicked item
            const rect = itemElement.getBoundingClientRect();
            // Adjust positioning relative to the iframe if needed, using window scroll offsets
            moveOptionsDiv.style.top = `${window.scrollY + rect.bottom}px`;
            moveOptionsDiv.style.left = `${window.scrollX + rect.left}px`;
            moveOptionsDiv.style.display = 'block';

            // Close options if clicking elsewhere
             document.addEventListener('click', closeMoveOptionsOnClickOutside, true);
        }

         function closeMoveOptionsOnClickOutside(event) {
            if (moveOptionsDiv.style.display === 'block' && !moveOptionsDiv.contains(event.target) && event.target !== currentItem) {
                moveOptionsDiv.style.display = 'none';
                document.removeEventListener('click', closeMoveOptionsOnClickOutside, true);
            }
        }


        function moveItem(targetSectionId) {
            if (!currentItem) return;
            const targetContainer = document.getElementById(`${targetSectionId}-items`);
            targetContainer.appendChild(currentItem); // Move the item
            currentItem.onclick = function() { // Update the onclick handler for the new section
                 showMoveOptions(this, targetSectionId);
            };
            moveOptionsDiv.style.display = 'none'; // Hide options
            currentItem = null;
             document.removeEventListener('click', closeMoveOptionsOnClickOutside, true);
        }

        function deleteItem() {
            if (!currentItem) return;
            currentItem.remove(); // Remove the item from the DOM
            moveOptionsDiv.style.display = 'none'; // Hide options
            currentItem = null;
            document.removeEventListener('click', closeMoveOptionsOnClickOutside, true);
        }

        // --- Example Data Loading ---
         function loadExampleData() {
             // Clear existing items first
             sections.forEach(id => { document.getElementById(`${id}-items`).innerHTML = ''; });

             document.getElementById('context-input').value = "Negotiating humanitarian access to IDP camp controlled by Armed Group Alpha. Key figure: Commander Khalid. Stated issues: Security concerns, need for notification. Underlying interests: Maintaining control, legitimacy, potential resource diversion.";

             addExampleItem('contested-facts', 'Number of armed personnel at checkpoint');
             addExampleItem('contested-facts', 'Origin of recent security incident');
             addExampleItem('contested-facts', 'Security situation on the access road');

             addExampleItem('agreed-facts', 'Medical needs exist in the camp');
             addExampleItem('agreed-facts', 'Local staff are from the community');
             addExampleItem('agreed-facts', 'Approximately 20,000 IDPs present');


             addExampleItem('divergent-norms', 'Humanitarian impartiality vs. local control');
             addExampleItem('divergent-norms', 'Transparency vs. operational security');
             addExampleItem('divergent-norms', 'International standards vs. local customs');


             addExampleItem('convergent-norms', 'Protection of vulnerable civilians');
             addExampleItem('convergent-norms', 'Respect for local community leadership');
             addExampleItem('convergent-norms', 'Desire to avoid large-scale health crisis');

         }

         function addExampleItem(section, text) {
             const itemsContainer = document.getElementById(`${section}-items`);
             const itemDiv = document.createElement('div');
             itemDiv.className = 'item';
             itemDiv.textContent = text;
             itemDiv.onclick = function() {
                 showMoveOptions(this, section);
             };
             itemsContainer.appendChild(itemDiv);
         }

         // Add example data load button on window load
         window.onload = function() {
             const contextSection = document.getElementById('context-section');
             const loadButton = document.createElement('button');
             loadButton.className = 'btn btn-outline-secondary mt-3';
             loadButton.textContent = 'Load Example Data';
             loadButton.onclick = loadExampleData;
             contextSection.appendChild(loadButton);
         };

    </script>
</body>
</html>
"""

# Integrated Assistant Dashboard HTML Structure (Simplified for Gradio embedding)
# We will use Gradio components to build the input part,
# and potentially Markdown/HTML for the output display structure.
# This HTML is more for reference or if we decide to embed parts.
assistant_dashboard_html_structure = """
    <div class="mb-3">
        <label for="scenarioInput" class="form-label">Negotiation Scenario:</label>
        <textarea class="form-control" id="scenarioInput" rows="8"></textarea>
    </div>
    <button id="analyzeButton" class="btn btn-primary">Analyze Scenario</button>
    <hr>
    <div class="card">
        <div class="card-header">
            <ul class="nav nav-tabs card-header-tabs" id="analysis-tabs" role="tablist">
                <li class="nav-item" role="presentation">
                    <button class="nav-link active" id="summary-tab" data-bs-toggle="tab" data-bs-target="#summary" type="button">Summary</button>
                </li>
                <li class="nav-item" role="presentation">
                    <button class="nav-link" id="stakeholders-tab" data-bs-toggle="tab" data-bs-target="#stakeholders" type="button">Stakeholders</button>
                </li>
                 <li class="nav-item" role="presentation">
                    <button class="nav-link" id="context-tab" data-bs-toggle="tab" data-bs-target="#context" type="button">Context</button>
                </li>
                 <li class="nav-item" role="presentation">
                    <button class="nav-link" id="analysis-tab" data-bs-toggle="tab" data-bs-target="#analysis" type="button">Framework Analysis</button>
                </li>
                 <li class="nav-item" role="presentation">
                    <button class="nav-link" id="risks-tab" data-bs-toggle="tab" data-bs-target="#risks" type="button">Risks</button>
                </li>
                <li class="nav-item" role="presentation">
                    <button class="nav-link" id="recommendations-tab" data-bs-toggle="tab" data-bs-target="#recommendations" type="button">Recommendations</button>
                </li>
            </ul>
        </div>
        <div class="card-body tab-content" id="analysis-tabs-content">
            <div class="tab-pane fade show active" id="summary" role="tabpanel"></div>
            <div class="tab-pane fade" id="stakeholders" role="tabpanel"></div>
            <div class="tab-pane fade" id="context" role="tabpanel"></div>
            <div class="tab-pane fade" id="analysis" role="tabpanel"></div>
            <div class="tab-pane fade" id="risks" role="tabpanel"></div>
            <div class="tab-pane fade" id="recommendations" role="tabpanel"></div>
        </div>
    </div>
"""

# Load MTP examples
mtp_examples = load_mtps_from_json()

# --- Gradio UI Construction ---
with gr.Blocks(theme=gr.themes.Soft(), title="LLM Toolkit for Humanitarian Negotiation") as app:
    gr.Markdown("# 🌍 LLM Toolkit for Humanitarian Negotiation")
    gr.Markdown("A comprehensive, interactive toolkit exploring LLMs in humanitarian negotiation, based on the provided notebook.")

    # --- Tab: Introduction & Setup ---
    with gr.Tab("👋 Introduction & Setup"):
        gr.Markdown("## Welcome to the Toolkit")
        gr.HTML("""
        <div style="background-color: #f5f5f5; padding: 20px; border-radius: 10px; border-left: 5px solid #3498db;">
          <h3 style="color: #555;">A practical and interactive guide for humanitarian negotiators</h3>
          <hr style="border-top: 1px solid #ddd;">
          <p style="font-size: 16px;">This interactive application is designed for humanitarian professionals who want to explore how <b>Large Language Models (LLMs)</b> can enhance their negotiation capabilities in challenging contexts.</p>
          <p style="font-size: 16px;">Based on the 'LLM Toolkit for Humanitarian Negotiation' notebook.</p>
        </div>
        """)
        gr.Markdown("## 🎯 Learning Objectives")
        gr.Markdown("""
        By using this toolkit, you will be able to:
        1.  **Identify** which LLMs are best suited for different aspects of humanitarian negotiation.
        2.  **Connect** to language models running locally (Ollama) for sensitive analysis and offline use.
        3.  **Trigger** automation workflows (n8n) for negotiation preparation and analysis.
        4.  **Generate** simple interfaces for negotiation-related projects using LLMs.
        5.  **Integrate** advanced tools like voice synthesis (ElevenLabs) for field briefings.
        6.  **Apply** this knowledge using an integrated Humanitarian Negotiation Assistant.
        7.  **Leverage** Mini Task Processes (MTPs) for structured prompts and outputs.
        8.  **Analyze** complex crisis scenarios using the Complete Crisis Framework.
        """)
        gr.Markdown("## 🔍 Case Studies")
        with gr.Accordion("IDP Camp Access Case", open=True):
            gr.Markdown(f"> {CASE_STUDY_EN}")

        with gr.Accordion("Edward University Campus Crisis", open=False):
            gr.Markdown(f"> {EDWARD_UNIVERSITY_CASE}")

        gr.Markdown("These scenarios will serve as our common thread for testing each tool and methodology.")
        gr.Markdown("---")
        gr.Markdown("### Initial Setup")
        gr.Markdown("Dependencies were installed when Cell 1 was executed. Ensure you have API keys ready for the cloud services you intend to use.")
        gr.Textbox(label="Current Date/Time", value=datetime.now().strftime("%Y-%m-%d %H:%M:%S"), interactive=False)

    # --- NEW Tab: Mini Task Processes (MTPs) ---
    with gr.Tab("🧩 Mini Task Processes (MTPs)"):
        gr.Markdown("## Mini Task Processes (MTPs)")
        gr.Markdown("MTPs allow non-technical users to generate structured prompts using pre-defined templates. Select a task type, input your scenario, and choose your preferred model.")

        # MTP Selection
        mtp_dropdown = gr.Dropdown(
            label="🔍 Select Task Type",
            choices=[mtp["title"] for mtp in mtp_examples],
            value=mtp_examples[0]["title"] if mtp_examples else None
        )
        mtp_description = gr.Markdown("*Select a task to see its description*")

        # Input area
        mtp_input = gr.Textbox(
            label="📝 Enter Scenario Text",
            placeholder="Enter your scenario or case study here...",
            lines=6
        )

        # Load examples
        load_case_study_btn_mtp1 = gr.Button("Load IDP Camp Case")
        load_case_study_btn_mtp2 = gr.Button("Load University Crisis Case")

        # API Keys
        with gr.Accordion("🔑 API Keys (Required for selected models)", open=False):
            with gr.Row():
                openai_key_mtp = gr.Textbox(label="OpenAI API Key", type="password")
                gemini_key_mtp = gr.Textbox(label="Google AI API Key", type="password")
                anthropic_key_mtp = gr.Textbox(label="Anthropic API Key", type="password")
                mistral_key_mtp = gr.Textbox(label="Mistral AI API Key", type="password")
                huggingface_key_mtp = gr.Textbox(label="Hugging Face API Key", type="password")

            with gr.Row():
                elevenlabs_key_mtp = gr.Textbox(label="ElevenLabs API Key", type="password")
                ollama_url_mtp = gr.Textbox(label="Ollama URL", value="http://localhost:11434")
                ollama_model_mtp = gr.Dropdown(
                    label="Ollama Model",
                    choices=["gemma:2b", "llama3:8b", "mistral:7b", "gemma:7b", "phi3:mini"],
                    value="gemma:2b"
                )

        # LLM Model Override (if user wants to use different model than recommended)
        with gr.Accordion("🤖 Model Selection", open=True):
            mtp_recommended_model = gr.Markdown("*Recommended model will appear here*")
            override_model = gr.Checkbox(label="Override recommended model", value=False)
            with gr.Column(visible=False) as override_model_col:
                selected_model = gr.Radio(
                    label="Select Model",
                    choices=["GPT-4 (OpenAI)", "Gemini (Google)", "Claude (Anthropic)", "Mistral (Mistral AI)",
                             "Hugging Face (Inference API)", "Ollama (Local)"],
                    value="Claude (Anthropic)"
                )

        # Output options
        with gr.Row():
            send_to_n8n_checkbox = gr.Checkbox(label="📤 Send results to n8n", value=False)
            speak_output_checkbox = gr.Checkbox(label="🔊 Convert output to speech", value=False)

        with gr.Row(visible=False) as n8n_settings:
            n8n_webhook_url_mtp = gr.Textbox(label="🔗 n8n Webhook URL", placeholder="Enter your n8n webhook URL")

        with gr.Row(visible=False) as voice_settings:
            voice_id_mtp = gr.Dropdown(
                label="🎙️ Voice",
                choices=["Rachel", "Adam", "Antoni", "Arnold", "Bella", "Domi", "Elli", "Josh", "Nicole", "Sarah"],
                value="Rachel"
            )

        # Processing button
        process_mtp_button = gr.Button("📋 Process Task", variant="primary")

        # Output areas
        mtp_output_text = gr.Textbox(label="Task Output", lines=12)
        mtp_audio_output = gr.Audio(label="Audio Output", type="filepath", visible=False)
        mtp_output_status = gr.Textbox(label="Status", lines=2)

        # --- MTP Logic ---

        # Display MTP description when selected
        def update_mtp_description(selected_mtp):
            for mtp in mtp_examples:
                if mtp["title"] == selected_mtp:
                    return f"**Description:** {mtp['description']}"
            return "*No description available*"

        def update_recommended_model(selected_mtp):
            for mtp in mtp_examples:
                if mtp["title"] == selected_mtp:
                    return f"**Recommended Model:** {mtp['preferred_model']}"
            return "*No model recommendation available*"

        def update_output_settings(selected_mtp):
            for mtp in mtp_examples:
                if mtp["title"] == selected_mtp:
                    send_to_n8n = mtp.get("send_to_n8n", False)
                    speak_output = mtp.get("speak_output", False)
                    return send_to_n8n, speak_output
            return False, False

        # Process MTP button logic
        def process_mtp(selected_mtp, input_text, openai_key, gemini_key, anthropic_key, mistral_key,
                        huggingface_key, elevenlabs_key, ollama_url, ollama_model, override_model_option,
                        selected_model_override, send_to_n8n, n8n_url, speak_output, voice_id):

            if not input_text:
                return "Please enter a scenario text.", None, "⚠️ Error: No input provided."

            # Get MTP template
            template = ""
            preferred_model = ""
            for mtp in mtp_examples:
                if mtp["title"] == selected_mtp:
                    template = mtp.get("prompt_template", "")
                    preferred_model = mtp.get("preferred_model", "Claude (Anthropic)")
                    break

            if not template:
                return "Error: Could not find template for selected task.", None, "⚠️ Error: Template not found."

            # Apply template
            prompt = template.replace("{input}", input_text)

            # Determine which model to use
            model_to_use = selected_model_override if override_model_option else preferred_model

            # Process with selected model
            start_time = time.time()
            result = "ERROR: Model processing failed."

            try:
                if model_to_use == "GPT-4 (OpenAI)":
                    if not openai_key:
                        return "ERROR: OpenAI API key required for this task.", None, "⚠️ Error: No OpenAI API key provided."
                    result = call_openai_api(openai_key, prompt)

                elif model_to_use == "Gemini (Google)":
                    if not gemini_key:
                        return "ERROR: Google AI API key required for this task.", None, "⚠️ Error: No Google AI API key provided."
                    result = call_gemini_api(gemini_key, prompt)

                elif model_to_use == "Claude (Anthropic)":
                    if not anthropic_key:
                        return "ERROR: Anthropic API key required for this task.", None, "⚠️ Error: No Anthropic API key provided."
                    result = call_claude_api(anthropic_key, prompt)

                elif model_to_use == "Mistral (Mistral AI)":
                    if not mistral_key:
                        return "ERROR: Mistral AI API key required for this task.", None, "⚠️ Error: No Mistral AI API key provided."
                    result = call_mistral_api(mistral_key, prompt)

                elif model_to_use == "Hugging Face (Inference API)":
                    if not huggingface_key:
                        return "ERROR: Hugging Face API key required for this task.", None, "⚠️ Error: No Hugging Face API key provided."
                    result = call_huggingface_api(huggingface_key, prompt)

                elif model_to_use == "Ollama (Local)":
                    result = call_ollama_api(ollama_model, prompt, ollama_url)

                else:
                    return f"ERROR: Unknown model type {model_to_use}", None, f"⚠️ Error: Unknown model {model_to_use}"

            except Exception as e:
                return f"ERROR: {str(e)}", None, f"⚠️ Error processing task: {str(e)}"

            processing_time = time.time() - start_time
            status = f"✅ Task processed in {processing_time:.2f} seconds using {model_to_use}"

            # Handle n8n if requested
            if send_to_n8n and n8n_url:
                try:
                    n8n_payload = json.dumps({
                        "task_type": selected_mtp,
                        "input": input_text,
                        "output": result,
                        "model": model_to_use,
                        "timestamp": datetime.now().isoformat()
                    })

                    n8n_response = trigger_n8n_webhook(n8n_url, n8n_payload)
                    status += f"\n{n8n_response}"
                except Exception as e:
                    status += f"\n❌ n8n Error: {str(e)}"

            # Handle voice synthesis if requested
            audio_path = None
            if speak_output:
                try:
                    if elevenlabs_key:
                        audio_path, voice_status = generate_elevenlabs_audio(elevenlabs_key, result, voice_id)
                        status += f"\n{voice_status}"
                    else:
                        # Fallback to local TTS
                        audio_path, voice_status = generate_local_tts(result)
                        status += f"\n{voice_status} (using local TTS)"
                except Exception as e:
                    status += f"\n❌ Voice synthesis error: {str(e)}"

            # Return results
            return result, audio_path, status

        # Connect UI components
        mtp_dropdown.change(
            fn=update_mtp_description,
            inputs=mtp_dropdown,
            outputs=mtp_description
        )

        mtp_dropdown.change(
            fn=update_recommended_model,
            inputs=mtp_dropdown,
            outputs=mtp_recommended_model
        )

        mtp_dropdown.change(
            fn=update_output_settings,
            inputs=mtp_dropdown,
            outputs=[send_to_n8n_checkbox, speak_output_checkbox]
        )

        override_model.change(
            fn=lambda x: gr.update(visible=x),
            inputs=override_model,
            outputs=override_model_col
        )

        send_to_n8n_checkbox.change(
            fn=lambda x: gr.update(visible=x),
            inputs=send_to_n8n_checkbox,
            outputs=n8n_settings
        )

        speak_output_checkbox.change(
            fn=lambda x: gr.update(visible=x),
            inputs=speak_output_checkbox,
            outputs=voice_settings
        )

        speak_output_checkbox.change(
            fn=lambda x: gr.update(visible=x),
            inputs=speak_output_checkbox,
            outputs=mtp_audio_output
        )

        load_case_study_btn_mtp1.click(lambda: CASE_STUDY_EN, inputs=[], outputs=mtp_input)
        load_case_study_btn_mtp2.click(lambda: EDWARD_UNIVERSITY_CASE, inputs=[], outputs=mtp_input)

        process_mtp_button.click(
            fn=process_mtp,
            inputs=[
                mtp_dropdown, mtp_input,
                openai_key_mtp, gemini_key_mtp, anthropic_key_mtp, mistral_key_mtp, huggingface_key_mtp,
                elevenlabs_key_mtp, ollama_url_mtp, ollama_model_mtp,
                override_model, selected_model,
                send_to_n8n_checkbox, n8n_webhook_url_mtp,
                speak_output_checkbox, voice_id_mtp
            ],
            outputs=[mtp_output_text, mtp_audio_output, mtp_output_status]
        )

        # Create new MTP section
        with gr.Accordion("➕ Create New Task Template", open=False):
            gr.Markdown("Create a new Mini Task Process (MTP) template for future use.")

            new_mtp_title = gr.Textbox(label="Title", placeholder="Enter a descriptive title for this task")
            new_mtp_description = gr.Textbox(label="Description", placeholder="Enter a brief description of what this task does")
            new_mtp_template = gr.Textbox(
                label="Prompt Template",
                placeholder="Enter the prompt template with {input} where the scenario text should go",
                lines=6
            )
            new_mtp_model = gr.Dropdown(
                label="Preferred Model",
                choices=["GPT-4 (OpenAI)", "Gemini (Google)", "Claude (Anthropic)", "Mistral (Mistral AI)",
                         "Hugging Face (Inference API)", "Ollama (Local)"],
                value="Claude (Anthropic)"
            )
            new_mtp_n8n = gr.Checkbox(label="Send to n8n by default", value=False)
            new_mtp_voice = gr.Checkbox(label="Generate voice by default", value=False)

            save_new_mtp_button = gr.Button("💾 Save New Task Template")
            new_mtp_status = gr.Textbox(label="Status", interactive=False)

            def save_mtp_template(title, description, template, model, n8n, voice):
                if not title or not template:
                    return "⚠️ Error: Title and template are required."

                new_mtp = {
                    "title": title,
                    "description": description,
                    "prompt_template": template,
                    "preferred_model": model,
                    "send_to_n8n": n8n,
                    "speak_output": voice
                }

                success = save_new_mtp(new_mtp)

                if success:
                    # Reload MTP examples
                    global mtp_examples
                    mtp_examples = load_mtps_from_json()
                    return f"✅ Successfully saved new task template: {title}"
                else:
                    return "❌ Error saving template. Check console for details."

            save_new_mtp_button.click(
                fn=save_mtp_template,
                inputs=[new_mtp_title, new_mtp_description, new_mtp_template,
                        new_mtp_model, new_mtp_n8n, new_mtp_voice],
                outputs=new_mtp_status
            )

    # --- NEW Tab: Crisis Negotiation Analysis Framework ---
    with gr.Tab("🔎 Crisis Analysis Framework"):
        gr.Markdown("## Crisis Negotiation Analysis Framework")
        gr.Markdown("""
        This comprehensive framework integrates 9 specialized analytical approaches to assess complex crisis negotiations:

        1. **Island of Agreements (IoA)** - Identify starting points for dialogue by mapping convergence/divergence
        2. **Iceberg Analysis** - Reveal underlying interests beneath stated positions
        3. **Stakeholder Matrix** - Map key actors based on power, legitimacy, and urgency
        4. **Influence Pathways** - Visualize key relationships and influence channels
        5. **Scenario Design** - Map negotiation position boundaries and identify potential agreement zones
        6. **Multiparty Dynamics** - Map complex stakeholder relationships and coalition-building opportunities
        7. **Tactical Timeline** - Structured approach to resolution through sequential phases
        8. **Radical Faction Analysis** - Identify potentially disruptive elements and risk management strategies
        9. **Success Factors** - Define critical elements needed for resolution
        """)

        # Input section
        crisis_scenario_input = gr.Textbox(
            label="📝 Enter Crisis Scenario",
            placeholder="Describe the crisis scenario in detail...",
            lines=8
        )

        # Load example buttons
        with gr.Row():
            load_case_study_btn_crisis1 = gr.Button("Load IDP Camp Case")
            load_case_study_btn_crisis2 = gr.Button("Load University Crisis Case")

        # API Keys
        with gr.Accordion("🔑 API Keys", open=False):
            with gr.Row():
                openai_key_crisis = gr.Textbox(label="OpenAI API Key", type="password")
                claude_key_crisis = gr.Textbox(label="Claude API Key", type="password")

        # Output format selection
        output_format = gr.Radio(
            label="Select Output Format",
            choices=["Text Analysis", "JSON Structured Output"],
            value="Text Analysis"
        )

        # Model selection
        model_choice_crisis = gr.Dropdown(
            label="Select LLM",
            choices=["GPT-4 (OpenAI)", "Claude (Anthropic)"],
            value="Claude (Anthropic)",
            info="We recommend Claude or GPT-4 for this complex analysis task."
        )

        # Analysis button
        analyze_crisis_button = gr.Button("🧠 Generate Comprehensive Analysis", variant="primary")

        # Results
        crisis_analysis_output = gr.Markdown("*Analysis results will appear here*")

        # Optional: JSON view for structured output
        with gr.Accordion("View/Copy JSON Output", open=False):
            json_output = gr.Code(
                label="JSON Output",
                language="json",
                interactive=True,
                lines=10
            )

        # Crisis Analysis Logic
        def perform_crisis_analysis(scenario, output_format, model_choice, openai_key, claude_key):
            if not scenario:
                return "⚠️ Please enter a crisis scenario to analyze.", "{}"

            # Build prompt based on output format
            prompt = build_crisis_analysis_prompt(scenario, output_format.lower().split()[0])

            # Select model and perform analysis
            result = "Error performing analysis."

            if model_choice == "GPT-4 (OpenAI)":
                if not openai_key:
                    return "❌ Error: OpenAI API key required for this analysis.", "{}"

                system_message = "You are a Crisis Negotiation Analyst specialized in comprehensive analytical breakdowns of complex humanitarian negotiation scenarios."
                result = call_openai_api(openai_key, prompt, model="gpt-4-turbo", system_message=system_message)

            elif model_choice == "Claude (Anthropic)":
                if not claude_key:
                    return "❌ Error: Claude API key required for this analysis.", "{}"

                system_message = "You are a Crisis Negotiation Analyst specialized in comprehensive analytical breakdowns of complex humanitarian negotiation scenarios."
                result = call_claude_api(claude_key, prompt, model="claude-3-sonnet-20240229", system_message=system_message)

            # Extract JSON if needed
            json_content = "{}"
            if output_format == "JSON Structured Output":
                try:
                    # Try to find and extract JSON from the result
                    json_start = result.find("{")
                    json_end = result.rfind("}")

                    if json_start >= 0 and json_end >= 0:
                        json_str = result[json_start:json_end+1]
                        # Validate JSON by parsing it
                        json_obj = json.loads(json_str)
                        # Format it nicely
                        json_content = json.dumps(json_obj, indent=2)
                    else:
                        json_content = '{"error": "No valid JSON found in response"}'
                except Exception as e:
                    json_content = f'{{"error": "Failed to parse JSON: {str(e)}"}}'

            return result, json_content

        # Connect UI components
        load_case_study_btn_crisis1.click(lambda: CASE_STUDY_EN, inputs=[], outputs=crisis_scenario_input)
        load_case_study_btn_crisis2.click(lambda: EDWARD_UNIVERSITY_CASE, inputs=[], outputs=crisis_scenario_input)

        analyze_crisis_button.click(
            fn=perform_crisis_analysis,
            inputs=[crisis_scenario_input, output_format, model_choice_crisis, openai_key_crisis, claude_key_crisis],
            outputs=[crisis_analysis_output, json_output]
        )

        # Show relevant sections of structured output
        output_format.change(
            fn=lambda x: gr.update(visible=(x == "JSON Structured Output")),
            inputs=output_format,
            outputs=json_output.parent
        )

    # --- Tab: LLM Platform Comparison ---
    with gr.Tab("🔄 LLM Platform Comparison"):
        gr.Markdown("## Compare Multiple LLM Platforms Side-by-Side")
        gr.Markdown("This tab allows you to directly compare responses from different LLM platforms on the same humanitarian negotiation prompt, helping you identify which platform is best suited for specific tasks.")

        # API Keys Section for all platforms
        with gr.Row():
            openai_key_comp = gr.Textbox(label="🔑 OpenAI API Key", type="password", placeholder="Enter your OpenAI API key (sk-...)")
            gemini_key_comp = gr.Textbox(label="🔑 Google AI (Gemini) API Key", type="password", placeholder="Enter your Google AI API key")
            claude_key_comp = gr.Textbox(label="🔑 Anthropic (Claude) API Key", type="password", placeholder="Enter your Anthropic API key")
            mistral_key_comp = gr.Textbox(label="🔑 Mistral AI API Key", type="password", placeholder="Enter your Mistral AI key")

        # Ollama settings (if needed for comparison)
        with gr.Row():
            ollama_url_comp = gr.Textbox(label="Ollama API URL (Local)", value="http://localhost:11434", placeholder="Usually http://localhost:11434")
            ollama_model_comp = gr.Dropdown(
                label="Ollama Model (Local)",
                choices=["llama3:8b", "mistral:7b", "gemma:7b", "phi3:mini"],
                value="llama3:8b",
                allow_custom_value=True
            )

        # Prompt and LLM Selection
        gr.Markdown("### Prompt & Models to Compare")
        prompt_input_comp = gr.Textbox(label="📝 Enter Prompt for Comparison", lines=7, placeholder="Enter your prompt here...")
        with gr.Row():
            load_case_study_btn_comp1 = gr.Button("Load IDP Camp Case")
            load_case_study_btn_comp2 = gr.Button("Load University Crisis Case")

        # Model selection (checkboxes so users can select multiple)
        gr.Markdown("### Select Models to Compare")
        with gr.Row():
            use_openai_check = gr.Checkbox(label="OpenAI (ChatGPT)", value=True)
            use_gemini_check = gr.Checkbox(label="Google (Gemini)", value=True)
            use_claude_check = gr.Checkbox(label="Anthropic (Claude)", value=True)
            use_mistral_check = gr.Checkbox(label="Mistral AI", value=False)
            use_ollama_check = gr.Checkbox(label="Ollama (Local)", value=False)

        # Optional framework application
        framework_choice_comp = gr.Dropdown(
            label="Apply Negotiation Framework to Prompt (Optional)",
            choices=["None", "Island of Agreement", "BATNA/ZOPA", "Iceberg Analysis", "Stakeholder Matrix", "Multi-party Dynamics", "Success Factors"],
            value="None"
        )

        # Run comparison button
        compare_button = gr.Button("🔍 Run Comparison", variant="primary")

        # Results display (using a DataFrame for side-by-side view)
        gr.Markdown("### Comparison Results")
        comparison_results = gr.Dataframe(
            headers=["Model", "Response", "Response Time (s)"],
            datatype=["str", "str", "number"],
            row_count=(5, "fixed"),
            col_count=(3, "fixed"),
            interactive=False,
        )

        # Detailed Response View (for when results table is too condensed)
        with gr.Accordion("Detailed Response View", open=False):
            selected_model_detail = gr.Dropdown(label="Select Model for Detailed View", choices=["OpenAI (ChatGPT)", "Google (Gemini)", "Anthropic (Claude)", "Mistral AI", "Ollama (Local)"])
            detailed_response = gr.Textbox(label="Detailed Response", lines=15, interactive=False)

        # Performance metrics visualization
        with gr.Accordion("Performance Metrics", open=False):
            metrics_html = gr.HTML("""
            <div id="metrics-placeholder">
                <p>Run a comparison to see performance metrics.</p>
                <p>Metrics will show response times and other data about the model responses.</p>
            </div>
            """)

        # --- Comparison Logic ---
        def run_llm_comparison(openai_key, gemini_key, claude_key, mistral_key,
                              ollama_url, ollama_model, prompt_text,
                              use_openai, use_gemini, use_claude, use_mistral, use_ollama,
                              framework):

            if not prompt_text:
                return ([["❌ Error", "Please enter a prompt to compare responses.", 0]],
                        [], "No comparison run yet.")

            # Apply framework to the prompt if selected
            processed_prompt = apply_negotiation_framework(framework, prompt_text)

            results = []
            responses = {}  # To store detailed responses for the dropdown view

            # OpenAI
            if use_openai:
                if not openai_key:
                    results.append(["OpenAI (ChatGPT)", "❌ Error: API key not provided", 0])
                    responses["OpenAI (ChatGPT)"] = "❌ Error: API key not provided"
                else:
                    start_time = time.time()
                    response = call_openai_api(openai_key, processed_prompt)
                    elapsed = time.time() - start_time

                    # Truncate response for the table display if too long
                    table_response = response[:500] + "..." if len(response) > 500 else response
                    results.append(["OpenAI (ChatGPT)", table_response, round(elapsed, 2)])
                    responses["OpenAI (ChatGPT)"] = response

            # Gemini
            if use_gemini:
                if not gemini_key:
                    results.append(["Google (Gemini)", "❌ Error: API key not provided", 0])
                    responses["Google (Gemini)"] = "❌ Error: API key not provided"
                else:
                    start_time = time.time()
                    response = call_gemini_api(gemini_key, processed_prompt)
                    elapsed = time.time() - start_time

                    table_response = response[:500] + "..." if len(response) > 500 else response
                    results.append(["Google (Gemini)", table_response, round(elapsed, 2)])
                    responses["Google (Gemini)"] = response

            # Claude
            if use_claude:
                if not claude_key:
                    results.append(["Anthropic (Claude)", "❌ Error: API key not provided", 0])
                    responses["Anthropic (Claude)"] = "❌ Error: API key not provided"
                else:
                    start_time = time.time()
                    response = call_claude_api(claude_key, processed_prompt)
                    elapsed = time.time() - start_time

                    table_response = response[:500] + "..." if len(response) > 500 else response
                    results.append(["Anthropic (Claude)", table_response, round(elapsed, 2)])
                    responses["Anthropic (Claude)"] = response

            # Mistral
            if use_mistral:
                if not mistral_key:
                    results.append(["Mistral AI", "❌ Error: API key not provided", 0])
                    responses["Mistral AI"] = "❌ Error: API key not provided"
                else:
                    start_time = time.time()
                    response = call_mistral_api(mistral_key, processed_prompt)
                    elapsed = time.time() - start_time

                    table_response = response[:500] + "..." if len(response) > 500 else response
                    results.append(["Mistral AI", table_response, round(elapsed, 2)])
                    responses["Mistral AI"] = response

            # Ollama (Local)
            if use_ollama:
                if not ollama_model:
                    results.append(["Ollama (Local)", "❌ Error: Model name not provided", 0])
                    responses["Ollama (Local)"] = "❌ Error: Model name not provided"
                else:
                    start_time = time.time()
                    response = call_ollama_api(ollama_model, processed_prompt, ollama_url)
                    elapsed = time.time() - start_time

                    table_response = response[:500] + "..." if len(response) > 500 else response
                    results.append(["Ollama (Local)", table_response, round(elapsed, 2)])
                    responses["Ollama (Local)"] = response

            # Create dropdown choices from actual models that were compared
            model_choices = [row[0] for row in results]

            # Generate performance metrics visualization
            if len(results) > 0:
                # Create a simple bar chart showing response times
                response_times = [row[2] for row in results]
                model_names = [row[0] for row in results]

                # Generate an HTML table with metrics
                metrics_table = f"""
                <div style="margin: 20px 0; max-width: 800px;">
                    <h4>Response Time Comparison</h4>
                    <table style="width: 100%; border-collapse: collapse; margin-top: 10px;">
                        <tr style="background-color: #f2f2f2;">
                            <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Model</th>
                            <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Response Time (s)</th>
                            <th style="padding: 8px; text-align: left; border: 1px solid #ddd;">Relative Speed</th>
                        </tr>
                """

                # Find the fastest model for relative comparison
                min_time = min([t for t in response_times if t > 0], default=1)

                for i, model in enumerate(model_names):
                    time_val = response_times[i]
                    if time_val > 0:
                        relative = f"{time_val/min_time:.1f}x"
                        # Color coding: green for fastest, yellow for medium, red for slowest
                        if time_val == min_time:
                            bg_color = "#e6ffe6"  # Light green
                        elif time_val < 2 * min_time:
                            bg_color = "#ffffcc"  # Light yellow
                        else:
                            bg_color = "#ffebe6"  # Light red
                    else:
                        relative = "N/A"
                        bg_color = "#f2f2f2"  # Light grey

                    metrics_table += f"""
                        <tr style="background-color: {bg_color};">
                            <td style="padding: 8px; text-align: left; border: 1px solid #ddd;">{model}</td>
                            <td style="padding: 8px; text-align: left; border: 1px solid #ddd;">{time_val:.2f}</td>
                            <td style="padding: 8px; text-align: left; border: 1px solid #ddd;">{relative}</td>
                        </tr>
                    """

                metrics_table += """
                    </table>
                    <p style="margin-top: 15px; font-style: italic;">
                        Note: Response times include API latency and may vary between runs.
                        Consider multiple comparisons for reliable benchmarks.
                    </p>
                </div>
                """
            else:
                metrics_table = "<p>No comparison data available.</p>"

            # If we have a detailed response for the first model, select it by default
            initial_detailed_response = responses.get(model_choices[0], "") if model_choices else ""

            return results, model_choices, initial_detailed_response, metrics_table, responses

        # Function to show detailed response
        def show_detailed_response(model, responses_dict):
            return responses_dict.get(model, "No response available for this model.")

        # Connect the load case study buttons
        load_case_study_btn_comp1.click(lambda: CASE_STUDY_EN, inputs=[], outputs=prompt_input_comp)
        load_case_study_btn_comp2.click(lambda: EDWARD_UNIVERSITY_CASE, inputs=[], outputs=prompt_input_comp)

        # Connect the comparison button
        results_state = gr.State({})  # State to store detailed responses

        compare_button.click(
            run_llm_comparison,
            inputs=[
                openai_key_comp, gemini_key_comp, claude_key_comp, mistral_key_comp,
                ollama_url_comp, ollama_model_comp, prompt_input_comp,
                use_openai_check, use_gemini_check, use_claude_check, use_mistral_check, use_ollama_check,
                framework_choice_comp
            ],
            outputs=[
                comparison_results,
                selected_model_detail,
                detailed_response,
                metrics_html,
                results_state
            ]
        )

        # Connect the detailed view dropdown
        selected_model_detail.change(
            show_detailed_response,
            inputs=[selected_model_detail, results_state],
            outputs=detailed_response
        )

        # Example prompts
        gr.Examples(
            examples=[
                ["Identify the key interests of Armed Group Alpha based on the case study."],
                ["Generate 3 opening statements that could be used to approach the checkpoint commander."],
                ["What legal frameworks apply to humanitarian access in this scenario?"],
                ["Create a one-page briefing on the negotiation strategy for field staff."]
            ],
            inputs=prompt_input_comp
        )

        # Comparison tips
        gr.Markdown("""
        ### Comparison Tips:

        - **Response Quality**: Compare depth of analysis, structure, relevance to humanitarian contexts
        - **Speed**: Note the response time differences (affected by model size and API latency)
        - **Framework Application**: See how different models interpret negotiation frameworks
        - **Ethical Considerations**: Note how models handle complex ethical dilemmas in negotiations
        - **Actionability**: Which model gives the most practical, implementable advice?
        """)

    # --- Tab: Cloud LLMs ---
    with gr.Tab("☁️ Cloud LLMs Exploration"):
        gr.Markdown("## Interact with Cloud-Based LLMs (Functional)")
        gr.Markdown("Test different LLMs with your prompts or the case study. **API Keys are required.**")

        with gr.Row():
            openai_key_input = gr.Textbox(label="🔑 OpenAI API Key", type="password", placeholder="Enter your OpenAI API key (sk-...)")
            gemini_key_input = gr.Textbox(label="🔑 Google AI (Gemini) API Key", type="password", placeholder="Enter your Google AI API key")
            anthropic_key_input = gr.Textbox(label="🔑 Anthropic (Claude) API Key", type="password", placeholder="Enter your Anthropic API key")
            mistral_key_input = gr.Textbox(label="🔑 Mistral AI API Key", type="password", placeholder="Enter your Mistral AI API key (optional)")

        with gr.Row():
             llm_choice_cloud = gr.Dropdown(
                label="Select LLM",
                choices=["ChatGPT (OpenAI)", "Gemini (Google)", "Claude (Anthropic)", "Mistral (Mistral AI)"],
                value="ChatGPT (OpenAI)"
            )
             framework_choice = gr.Dropdown(
                label="Apply Negotiation Framework (Optional)",
                choices=["None", "Island of Agreement", "BATNA/ZOPA", "Iceberg Analysis", "Stakeholder Matrix", "Multi-party Dynamics", "Success Factors"],
                value="None"
            )

        prompt_input_cloud = gr.Textbox(label="📝 Enter Prompt", lines=7, placeholder="Enter your prompt here, or use the case study text.")
        with gr.Row():
            load_case_study_btn_cloud1 = gr.Button("Load IDP Camp Case")
            load_case_study_btn_cloud2 = gr.Button("Load University Crisis Case")
        submit_button_cloud = gr.Button("🚀 Send Prompt to Cloud LLM", variant="primary")
        output_cloud = gr.Textbox(label="LLM Response", lines=15, interactive=False)

        # --- Cloud LLM Logic ---
        def cloud_llm_interface(api_openai, api_gemini, api_anthropic, api_mistral, choice, framework, prompt):
            if not prompt:
                 return "⚠️ Please enter a prompt."

            # Apply framework to the prompt if selected
            processed_prompt = apply_negotiation_framework(framework, prompt)
            if framework.lower() != 'none':
                 print(f"ℹ️ Applying framework '{framework}'. Modified prompt:\n{processed_prompt[:300]}...") # Log modified prompt

            result = f"❌ Error: LLM choice '{choice}' not recognized." # Default error

            if choice == "ChatGPT (OpenAI)":
                result = call_openai_api(api_openai, processed_prompt)
            elif choice == "Gemini (Google)":
                result = call_gemini_api(api_gemini, processed_prompt)
            elif choice == "Claude (Anthropic)":
                result = call_claude_api(api_anthropic, processed_prompt)
            elif choice == "Mistral (Mistral AI)":
                result = call_mistral_api(api_mistral, processed_prompt)

            return result

        load_case_study_btn_cloud1.click(lambda: CASE_STUDY_EN, inputs=[], outputs=prompt_input_cloud)
        load_case_study_btn_cloud2.click(lambda: EDWARD_UNIVERSITY_CASE, inputs=[], outputs=prompt_input_cloud)

        submit_button_cloud.click(
            cloud_llm_interface,
            inputs=[openai_key_input, gemini_key_input, anthropic_key_input, mistral_key_input, llm_choice_cloud, framework_choice, prompt_input_cloud],
            outputs=output_cloud
        )

        gr.Examples(
            examples=[
                ["ChatGPT (OpenAI)", "None", "Based on the case study, what are the 3 main interests of Armed Group Alpha?"],
                ["Gemini (Google)", "Island of Agreement", CASE_STUDY_EN],
                ["Claude (Anthropic)", "BATNA/ZOPA", "Scenario: Negotiating release of detained staff member. Org wants immediate release. Counterpart wants prisoner exchange."],
                ["Mistral (Mistral AI)", "None", "Draft a neutral opening statement for the first meeting with Armed Group Alpha based on the case study."]
            ],
            inputs=[llm_choice_cloud, framework_choice, prompt_input_cloud],
            outputs=output_cloud,
            fn=cloud_llm_interface, # Function to call for examples
            cache_examples=False # Avoid caching API calls
        )
        gr.Markdown("---")
        gr.Markdown("### Benchmarking & Frameworks")
        gr.Markdown("The 'Apply Negotiation Framework' dropdown modifies your prompt to ask the LLM for specific analysis structures (IoA, BATNA/ZOPA). Live benchmarking between models is complex; evaluate responses based on relevance, coherence, and actionable insights for your negotiation needs.")


    # --- Tab: Local LLMs (Ollama) ---
    with gr.Tab("💻 Local LLMs (Ollama Connection)"):
        gr.Markdown("## Connect to Local LLMs via Ollama (Functional)")
        gr.Markdown("**Requirement:** You MUST have Ollama installed and running on your local machine for this tab to work.")
        gr.HTML("""
        <ul>
            <li>Download and install Ollama from <a href='https://ollama.com/' target='_blank'>ollama.com</a>.</li>
            <li>Pull models using the terminal: <code>ollama pull model_name</code> (e.g., <code>ollama pull llama3:8b</code>, <code>ollama pull mistral:7b</code>, <code>ollama pull gemma:2b</code>).</li>
            <li>Ensure Ollama is running in the background.</li>
            <li><b>Connection Issues?</b> If Gradio is running in Colab, connecting to <code>localhost:11434</code> might fail. Consider running this Gradio script locally on your machine, or setting up tunneling (advanced).</li>
        </ul>
        """)
        with gr.Row():
            # Common Ollama models - user can also type a custom one
            ollama_model_input = gr.Dropdown(
                label="Select or Enter Local Ollama Model Name",
                choices=["llama3:8b", "mistral:7b", "gemma:2b", "gemma:7b", "phi3:mini"],
                value="gemma:2b",
                allow_custom_value=True
            )
            ollama_url_input = gr.Textbox(label="Ollama API URL", value="http://localhost:11434", placeholder="Usually http://localhost:11434")

        prompt_input_ollama = gr.Textbox(label="📝 Enter Prompt for Local LLM", lines=7, placeholder="Enter your prompt here...")
        with gr.Row():
            load_case_study_btn_ollama1 = gr.Button("Load IDP Camp Case")
            load_case_study_btn_ollama2 = gr.Button("Load University Crisis Case")
        submit_button_ollama = gr.Button("🚀 Send Prompt to Local Ollama", variant="primary")
        output_ollama = gr.Textbox(label="Ollama Response", lines=15, interactive=False)

        # --- Ollama Logic ---
        def ollama_interface(model, url, prompt):
            if not prompt:
                return "⚠️ Please enter a prompt."
            if not model:
                 return "❌ Please select or enter an Ollama model name."
            if not url:
                 return "❌ Please enter the Ollama API URL."

            # Try the API method first
            try:
                result = call_ollama_api(model, prompt, url)
                # If result starts with error message, try subprocess method as fallback
                if result.startswith("❌"):
                    print("API method failed, trying subprocess method...")
                    subprocess_result = call_ollama_subprocess(model, prompt)
                    if not subprocess_result.startswith("❌"):
                        return subprocess_result + "\n\n(Note: Used subprocess method as API method failed)"
                return result
            except Exception as e:
                # If API method fails completely, try subprocess
                print(f"API method failed with exception: {e}, trying subprocess method...")
                try:
                    return call_ollama_subprocess(model, prompt)
                except Exception as sub_e:
                    return f"❌ Both API and subprocess methods failed.\nAPI error: {e}\nSubprocess error: {sub_e}"

        load_case_study_btn_ollama1.click(lambda: CASE_STUDY_EN, inputs=[], outputs=prompt_input_ollama)
        load_case_study_btn_ollama2.click(lambda: EDWARD_UNIVERSITY_CASE, inputs=[], outputs=prompt_input_ollama)

        submit_button_ollama.click(
            ollama_interface,
            inputs=[ollama_model_input, ollama_url_input, prompt_input_ollama],
            outputs=output_ollama
        )

        gr.Examples(
            examples=[
                ["gemma:2b", "http://localhost:11434", "What are 3 key negotiation strategies I could use with Armed Group Alpha?"],
                ["llama3:8b", "http://localhost:11434", "Summarize the key challenges in the provided case study."],
                ["mistral:7b", "http://localhost:11434", "Generate 3 potential questions to ask Armed Group Alpha to understand their security concerns better."]
            ],
            inputs=[ollama_model_input, ollama_url_input, prompt_input_ollama],
            outputs=output_ollama,
            fn=ollama_interface,
            cache_examples=False # Ollama responses can vary
        )
        gr.Markdown("---")
        gr.Markdown("### Alternative Local Tools")
        gr.Markdown("The original notebook also mentions **AnythingLLM**. It provides a user interface for interacting with local models and documents, often using Ollama or other backends. Explore it at [useanything.com](https://useanything.com/).")


    # --- Tab: Automation (n8n) ---
    with gr.Tab("⚙️ Automation (n8n Webhook Trigger)"):
        gr.Markdown("## Trigger n8n Automation Workflows (Functional)")
        gr.Markdown("**Requirement:** You need an n8n instance (cloud or self-hosted) with a workflow set up to be triggered by a Webhook node.")
        gr.HTML("""
        <ul>
            <li>Create a workflow in n8n.</li>
            <li>Add a 'Webhook' node as the trigger.</li>
            <li>Copy the 'Test URL' or 'Production URL' provided by the Webhook node.</li>
            <li>Ensure your n8n instance is running and accessible from where you run this script.</li>
        </ul>
        <p>You can use this to send data (e.g., analysis results, scenario summaries) from this toolkit to n8n to trigger further actions like updating databases, sending notifications, etc.</p>
        """)

        n8n_webhook_url_input = gr.Textbox(label="🔗 n8n Webhook URL", placeholder="Paste your n8n webhook Test or Production URL here")
        n8n_payload_input = gr.Textbox(label="📄 Payload Data (JSON format)", lines=8, value='{\n  "source": "LLM Toolkit Gradio App",\n  "event": "Scenario Analysis Completed",\n  "data": {\n    "scenario_summary": "Negotiating access for IDP camp...",\n    "key_finding": "High mistrust level detected."\n  },\n  "timestamp": "' + datetime.now().isoformat() + '"\n}')
        submit_button_n8n = gr.Button("🚀 Trigger n8n Webhook", variant="primary")
        output_n8n = gr.Textbox(label="n8n Response / Status", lines=5, interactive=False)

        # --- n8n Logic ---
        def n8n_interface(url, payload):
            if not url:
                return "⚠️ Please enter the n8n webhook URL."
            if not payload:
                 return "⚠️ Please enter the JSON payload."

            # Call the actual n8n trigger function
            result = trigger_n8n_webhook(url, payload)
            return result

        submit_button_n8n.click(
            n8n_interface,
            inputs=[n8n_webhook_url_input, n8n_payload_input],
            outputs=output_n8n
        )
        gr.Markdown("---")
        gr.Markdown("### Automation Ideas")
        gr.Markdown("Connect LLM analysis results to: update a negotiation tracker (Airtable, Sheets), send summaries via Email/Slack, create tasks in project management tools, generate draft reports.")
        gr.Markdown("Other automation platforms like **Make.com** and **Zapier** offer similar webhook capabilities.")


    # --- Tab: Interface Generation ---
    with gr.Tab("🎨 Interface Generation"):
        gr.Markdown("## Generating Interfaces with LLMs & Examples")
        gr.Markdown("LLMs can help generate code for simple web interfaces. You can use the Cloud LLMs tab to ask models (like Claude, GPT-4, Gemini) to create HTML, CSS, and JavaScript for specific tools.")

        gr.Markdown("### Example Prompts for LLMs")
        with gr.Accordion("Show Example Prompts", open=False):
            gr.Code(language="markdown", value="""
**Prompt 1: Negotiation Planning Tool**
"Create a simple HTML page using Bootstrap 5 CSS. It should have input fields for: Negotiation Goal, Key Counterparts, My BATNA, Their Estimated BATNA, Potential Options. Include a button to save or print the plan (basic JS alert is fine)."

**Prompt 2: Stakeholder Mapping Interface**
"Generate HTML and CSS for a stakeholder mapping tool. Use cards for each stakeholder. Each card should have fields for: Name, Role, Stated Position, Likely Interests, Influence Level (1-5), Relationship (Ally/Neutral/Blocker). Allow adding new stakeholder cards dynamically using JavaScript."

**Prompt 3: Island of Agreement Tool (like the one below)**
"Create an HTML page using Bootstrap 5 for the 'Island of Agreement' negotiation framework. It needs four columns: Contested Facts, Agreed Facts, Divergent Norms/Interests, Convergent Norms/Interests. Each column should allow adding text items. Items should be movable between columns and deletable using JavaScript."
            """)

        gr.Markdown("### Example: Island of Agreement Tool (Embedded)")
        gr.Markdown("Below is the interactive 'Island of Agreement' tool generated based on the notebook's code, embedded directly here.")
        gr.HTML(ioa_interface_html) # Embed the HTML interface

        gr.Markdown("---")
        gr.Markdown("### Code for the IoA Tool Above")
        with gr.Accordion("Show HTML/CSS/JS Code", open=False):
             gr.Code(language="html", value=ioa_interface_html)


    # --- Tab: Voice Synthesis (ElevenLabs) ---
    with gr.Tab("🎙️ Voice Synthesis (ElevenLabs)"):
        gr.Markdown("## Generate Audio Briefings with ElevenLabs (Functional)")
        gr.Markdown("Convert text to speech for briefings, accessibility, or multilingual communication. **Requires an ElevenLabs API Key.**")

        elevenlabs_api_key_input = gr.Textbox(label="🔑 ElevenLabs API Key", type="password", placeholder="Enter your ElevenLabs API key")
        # Common voices - check ElevenLabs website for full list and IDs if needed
        elevenlabs_voice_id_input = gr.Dropdown(
            label="Select Voice (Name or ID)",
            choices=["Rachel", "Adam", "Antoni", "Arnold", "Bella", "Domi", "Elli", "Josh", "Nicole", "Sarah", "Jeremy"], # Add more or use IDs
            value="Rachel",
            allow_custom_value=True # Allow entering voice IDs directly
        )
        text_to_speak_input = gr.Textbox(label="📝 Text to Synthesize", lines=8, placeholder="Enter the text to convert to audio...")

        with gr.Row():
            load_case_study_btn_eleven1 = gr.Button("Load IDP Camp Briefing")
            load_case_study_btn_eleven2 = gr.Button("Load University Crisis Briefing")

        # Sample briefings
        idp_briefing = """
        Field Security Briefing: Northern Province IDP Camp Access

        Current situation: Armed Group Alpha controls checkpoints around the IDP camp. Approximately 20,000 displaced civilians require urgent humanitarian assistance including medical care, food, and water.

        Key contacts: Commander Khalid is the primary checkpoint authority. Local community leaders Fatima Hassan and Ibrahim Nur can facilitate communication.

        Security protocols: All vehicles must display organizational emblems. Staff must carry identification. No photos at checkpoints. Maintain radio contact with base at all times.

        Contingency plan: If access is denied, return to base immediately and report to security coordinator. Do not attempt negotiation without proper authorization.
        """

        university_briefing = """
        Security Update: Edward University Campus Crisis

        Current situation: Student occupation of administration building continues with increased tensions. External activists have joined protests. Media presence is extensive.

        Key parties: University administration seeks immediate evacuation. Moderate student leaders willing to negotiate. External activists pursuing broader agenda.

        Security protocols: Maintain neutral stance. Avoid crossing police lines. Carry identification at all times. Do not engage with media without authorization.

        Authorized contacts: Dean Roberts is our primary university contact. Student representative Sarah Chen is moderate faction liaison. Avoid direct engagement with radical elements.
        """

        # Voice settings
        with gr.Accordion("Voice Settings", open=False):
            stability_slider = gr.Slider(minimum=0.0, maximum=1.0, value=0.7, step=0.1, label="Stability")
            similarity_boost_slider = gr.Slider(minimum=0.0, maximum=1.0, value=0.7, step=0.1, label="Similarity Boost")

        generate_audio_button = gr.Button("🔊 Generate Audio", variant="primary")

        audio_output = gr.Audio(label="Generated Audio", type="filepath") # Use filepath to handle the generated file
        status_output_eleven = gr.Textbox(label="Status", lines=2, interactive=False)

        # Add fallback option for local TTS
        use_local_tts = gr.Checkbox(label="Use local TTS (if no ElevenLabs API key)", value=False)

        # --- ElevenLabs Logic ---
        def elevenlabs_interface(api_key, voice, text, stability, similarity_boost, use_local):
             if not text:
                 return None, "⚠️ Please enter text to synthesize."

             # Check if local TTS fallback should be used
             if not api_key or use_local:
                 # Use local TTS
                 audio_file_path, status_msg = generate_local_tts(text)
                 return audio_file_path, status_msg

             # Call the ElevenLabs function with enhanced parameters
             audio_file_path, status_msg = generate_elevenlabs_audio(
                 api_key, text, voice, stability=stability, similarity_boost=similarity_boost
             )
             return audio_file_path, status_msg

        load_case_study_btn_eleven1.click(lambda: idp_briefing, inputs=[], outputs=text_to_speak_input)
        load_case_study_btn_eleven2.click(lambda: university_briefing, inputs=[], outputs=text_to_speak_input)

        generate_audio_button.click(
            elevenlabs_interface,
            inputs=[elevenlabs_api_key_input, elevenlabs_voice_id_input, text_to_speak_input,
                   stability_slider, similarity_boost_slider, use_local_tts],
            outputs=[audio_output, status_output_eleven]
        )

        gr.Examples(
            examples=[
                ["Rachel", "Briefing summary for tomorrow's meeting: Confirm security protocols and beneficiary lists."],
                ["Adam", "Key negotiation points: Unrestricted access, respect for humanitarian principles, staff protection."]
            ],
            inputs=[elevenlabs_voice_id_input, text_to_speak_input],
            outputs=[audio_output, status_output_eleven],
            fn=lambda voice, text: elevenlabs_interface("", voice, text, 0.7, 0.7, True),  # Use local TTS for examples
            cache_examples=False
        )
        gr.Markdown("---")
        gr.Markdown("### Integration Ideas")
        gr.Markdown("""
        * **Pre-meeting Briefings:** Audio versions of prep docs for listening on the go.
        * **Multilingual Comms:** Convert updates to audio in team members' languages.
        * **Accessibility:** Hands-free access to analysis for field staff.
        * **Automation:** Connect via n8n to auto-generate audio when analysis is ready.
        """)


    # --- Tab: Integrated Assistant ---
    with gr.Tab("🤝 Integrated Negotiation Assistant"):
        gr.Markdown("## Humanitarian Negotiation Assistant (Functional)")
        gr.Markdown("Get comprehensive analysis of a negotiation scenario using a selected LLM. **API Keys required for Cloud/Local LLMs.**")

        gr.Markdown("### 1. Input Scenario & Choose LLM")
        with gr.Row():
             assistant_llm_choice = gr.Dropdown(
                label="Select Analysis LLM",
                choices=["ChatGPT (OpenAI)", "Gemini (Google)", "Claude (Anthropic)", "Mistral (Mistral AI)", "Ollama (Local)"],
                value="ChatGPT (OpenAI)"
            )
             # Input for Ollama model needed only if Ollama is chosen
             assistant_ollama_model = gr.Textbox(label="Ollama Model Name (if Local)", placeholder="e.g., llama3:8b", visible=False) # Initially hidden

        scenario_input_assistant = gr.Textbox(label="📝 Negotiation Scenario", lines=10, value=CASE_STUDY_EN)

        with gr.Row():
            load_case_study_btn_asst1 = gr.Button("Load IDP Camp Case")
            load_case_study_btn_asst2 = gr.Button("Load University Crisis Case")

        gr.Markdown("### 2. Provide API Keys (if needed)")
        with gr.Accordion("API Keys (Required for selected LLM)", open=False):
             with gr.Row():
                openai_key_input_asist = gr.Textbox(label="🔑 OpenAI API Key", type="password")
                gemini_key_input_asist = gr.Textbox(label="🔑 Gemini API Key", type="password")
                anthropic_key_input_asist = gr.Textbox(label="🔑 Claude API Key", type="password")
                mistral_key_input_asist = gr.Textbox(label="🔑 Mistral API Key", type="password")
                # No key needed for Ollama, but URL might be relevant
                ollama_url_input_asist = gr.Textbox(label="Ollama API URL (if Local)", value="http://localhost:11434")

        # Add analysis framework selection
        assistant_framework = gr.Radio(
            label="🧠 Analysis Framework",
            choices=["Basic Analysis", "Island of Agreement", "BATNA/ZOPA", "Complete Crisis Framework"],
            value="Basic Analysis",
            info="Choose depth of analysis. Complete Crisis Framework works best with Claude or GPT-4."
        )

        analyze_button_assistant = gr.Button("💡 Analyze Scenario", variant="primary")

        gr.Markdown("### 3. Analysis Results")
        # Use Markdown for structured output, potentially with HTML for better formatting if needed
        analysis_output_assistant = gr.Markdown(label="Scenario Analysis")

        # --- Assistant Logic ---
        def assistant_interface(llm_choice, ollama_model, scenario, api_openai, api_gemini, api_anthropic, api_mistral, ollama_url, framework_choice):
            if not scenario:
                return "⚠️ Please enter a negotiation scenario."

            # Build the structured prompt based on selected framework
            if framework_choice == "Basic Analysis":
                structured_prompt = build_assistant_prompt(scenario)
            elif framework_choice == "Island of Agreement":
                structured_prompt = apply_negotiation_framework("Island of Agreement", scenario)
            elif framework_choice == "BATNA/ZOPA":
                structured_prompt = apply_negotiation_framework("BATNA/ZOPA", scenario)
            elif framework_choice == "Complete Crisis Framework":
                structured_prompt = build_crisis_analysis_prompt(scenario, "text")
            else:
                # Default to basic analysis
                structured_prompt = build_assistant_prompt(scenario)

            result = f"❌ Error: LLM choice '{llm_choice}' not recognized." # Default error

            if llm_choice == "ChatGPT (OpenAI)":
                system_message = "You are a humanitarian negotiation expert specializing in crisis analysis and strategic engagement."
                result = call_openai_api(api_openai, structured_prompt, system_message=system_message)
            elif llm_choice == "Gemini (Google)":
                result = call_gemini_api(api_gemini, structured_prompt)
            elif llm_choice == "Claude (Anthropic)":
                system_message = "You are a humanitarian negotiation expert specializing in crisis analysis and strategic engagement."
                result = call_claude_api(api_anthropic, structured_prompt, system_message=system_message)
            elif llm_choice == "Mistral (Mistral AI)":
                result = call_mistral_api(api_mistral, structured_prompt)
            elif llm_choice == "Ollama (Local)":
                 if not ollama_model:
                     return "❌ Please enter the Ollama Model Name when 'Ollama (Local)' is selected."
                 result = call_ollama_api(ollama_model, structured_prompt, ollama_url)

            # Return result formatted as Markdown
            return result

        # Show/hide Ollama model input based on LLM choice
        def toggle_ollama_input(choice):
            if choice == "Ollama (Local)":
                return gr.update(visible=True)
            else:
                return gr.update(visible=False)

        assistant_llm_choice.change(toggle_ollama_input, inputs=assistant_llm_choice, outputs=assistant_ollama_model)

        load_case_study_btn_asst1.click(lambda: CASE_STUDY_EN, inputs=[], outputs=scenario_input_assistant)
        load_case_study_btn_asst2.click(lambda: EDWARD_UNIVERSITY_CASE, inputs=[], outputs=scenario_input_assistant)

        analyze_button_assistant.click(
            assistant_interface,
            inputs=[
                assistant_llm_choice, assistant_ollama_model, scenario_input_assistant,
                openai_key_input_asist, gemini_key_input_asist, anthropic_key_input_asist, mistral_key_input_asist,
                ollama_url_input_asist, assistant_framework
            ],
            outputs=analysis_output_assistant
        )


    # --- Tab: Resources & Glossary ---
    with gr.Tab("📚 Resources & Glossary"):
        gr.Markdown("## Additional Resources and Glossary")
        # Add content from the notebook's resource section here
        gr.Markdown("""
        ### Key Terms Glossary
        * **LLM (Large Language Model):** AI model trained on vast text data to understand and generate human-like language (e.g., GPT-4, Gemini, Claude, Llama).
        * **API (Application Programming Interface):** A way for different software programs to communicate with each other. Used here to access cloud LLMs.
        * **Cloud LLM:** Models hosted by companies (OpenAI, Google, Anthropic) accessed via the internet/API.
        * **Local LLM:** Models run entirely on your own computer, offering privacy and offline capability.
        * **Ollama:** A tool that simplifies running local LLMs.
        * **MTP (Mini Task Process):** Structured prompt templates with predefined parameters for consistent outputs.
        * **n8n / Make / Zapier:** Workflow automation platforms to connect different apps and services.
        * **Webhook:** A web address that can receive data, often used to trigger automation workflows.
        * **Gradio / Streamlit:** Python libraries for building simple web interfaces for data science and AI applications.
        * **ElevenLabs:** A platform for realistic AI voice synthesis (Text-to-Speech).
        * **Prompt Engineering:** The art and science of crafting effective inputs (prompts) to get desired outputs from LLMs.
        * **BATNA (Best Alternative To a Negotiated Agreement):** What a party will do if they fail to reach an agreement.
        * **ZOPA (Zone Of Possible Agreement):** The range where an agreement is possible that is acceptable to all parties.
        * **Island of Agreement:** A framework focusing on identifying common ground (agreed facts, convergent norms) alongside differences (contested facts, divergent norms).
        * **Iceberg Analysis:** Technique to discover underlying interests beneath stated positions in negotiations.
        * **Stakeholder Matrix:** Tool for mapping actors by their power, legitimacy, and urgency in a crisis.
        * **Complete Crisis Framework:** A comprehensive 9-part analytical approach for complex humanitarian crisis negotiations.

        ### Useful Links
        * [Ollama](https://ollama.com/)
        * [n8n.io](https://n8n.io/)
        * [OpenAI API](https://platform.openai.com/)
        * [Google AI Studio (Gemini)](https://aistudio.google.com/)
        * [Anthropic (Claude)](https://www.anthropic.com/)
        * [Mistral AI](https://mistral.ai/)
        * [ElevenLabs](https://elevenlabs.io/)
        * [Gradio](https://www.gradio.app/)
        * [AnythingLLM](https://useanything.com/)
        * [CCHN Field Manual](https://frontline-negotiations.org/resources/cchn-field-manual/)
        * [Humanitarian Negotiation Toolkit](https://www.alnap.org/help-library/humanitarian-negotiation-a-handbook-for-securing-access-assistance-and-protection-for)
        * [Island of Agreement Framework](https://www.usip.org/publications/2010/10/negotiating-forward-consensus-building)
        * [Harvard Negotiation Project](https://www.pon.harvard.edu/research_projects/harvard-negotiation-project/)
        """)

        gr.Markdown("### Recommended YouTube Channels")
        gr.Markdown("""
        * **Fireship 🔥** – Quick tutorials on Ollama, Gemini, and modern AI tools: [Fireship](https://www.youtube.com/c/Fireship)
        * **CodeEmporium** – Great tutorials on ElevenLabs API and voice synthesis: [CodeEmporium](https://www.youtube.com/channel/UC5k_PVN_SYcNBTSQ-TtCnpQ)
        * **AssemblyAI** – Excellent content on voice AI and NLP tools: [AssemblyAI](https://www.youtube.com/c/AssemblyAI)
        * **Data School** – Tutorials on n8n automation and Hugging Face: [Data School](https://www.youtube.com/c/dataschool)
        * **Humanitarian Negotiations** – Harvard's Program on Negotiation: [PON](https://www.youtube.com/playlist?list=PLvSHJahq4Z-T9yEr3I1b5wDdKP-VBxcYY)
        """)

print("✅ Gradio interface created.")


# @title Cell 4: Launch Gradio Application
# ---
# Executes this cell to start the Gradio application.
# A public link will be generated (if share=True) to access the interface.
# ---
print("🚀 Launching Gradio application...")
# share=True generates a public temporary link (valid for ~72h)
# debug=True shows more detailed error messages in the console
app.launch(share=True, debug=True)

⏳ Installing dependencies...
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 89.9/89.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.0/62.0 kB 956.2 kB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.5/46.5 MB 18.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 322.2/322.2 kB 19.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 243.4/243.4 kB 16.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 413.7/413.7 kB 21.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 288.7/288.7 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 74.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 16.4/16.4 MB 55.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.6/8.6 MB 57.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 95.2/95.2 kB 5.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.3/11.3 MB 68.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━